## Set up

### Import libraries

In [42]:
import arcpy
import pandas as pd

### Make a feature class from the layer package

In [43]:
# Define file path to layer package and geodatabase; file suffix
FundingDist_lpkx = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\FundingDistribution_AR25.lpkx"
project_gdb = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Funding_Distribution_AR25.gdb"
suffix = "AR25"

# Define the target feature class path in the project geodatabase where you would like to export features to
Funding_Dist = f"{project_gdb}\\Funding_Dist_{suffix}"

# Make a feature layer from the layer package (.lpkx)
arcpy.management.MakeFeatureLayer(FundingDist_lpkx, "temp_layer")

# Clear any existing selection
arcpy.management.SelectLayerByAttribute("temp_layer", "CLEAR_SELECTION")

# Ensure all features will be included
count = int(arcpy.management.GetCount("temp_layer")[0])
print(f"Number of features in source: {count}")

# Export the feature layer to your project geodatabase
arcpy.management.CopyFeatures("temp_layer", Funding_Dist)

# Optional cleanup if needed (delete the temporary layer)
arcpy.management.Delete("temp_layer")

print(f"Feature class has been successfully exported to: {project_gdb}")

Number of features in source: 10278
Feature class has been successfully exported to: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Funding_Distribution_AR25.gdb


## Remove Unneeded Fields

In [44]:
fields_to_remove = ["CIscore", 'Asthma', 'Cardiovascular', 'Cleanups', 'Diesel_PM', 'Drinking_Water', 'Education', 'FID_CES4PP_ctAR25', 
                    'FID_PP_AR25', 'GroundwaterThreats', 'Haz_Waste', 'HousBurd', 'Imp_Water_Bodies', 'Lead', 'Ling_Isol',
                    'Low_Birth_Weight', 'Ozone', 'Pesticides', 'PM2_5', 'Point_Count', 'Pollution', 'PollutionScore', 'PopChar', 
                    'PopCharScore', 'Poverty', 'Solid_Waste', 'Tox_Releases', 'Traffic', 'Unemployment', "point_noppggrf_total", 
                    "Poly_NOPPGGRF_TOTAL", "CT_NOPPGGRF_TOTAL", "Line_NOPPGGRF_TOTAL", "point_ppggrf_total", "Poly_PPGGRF_TOTAL", 
                    "CT_PPGGRF_TOTAL", "Line_PPGGRF_TOTAL", "point_ggrf_total", "Poly_GGRF_TOTAL", "CT_GGRF_TOTAL", 
                    "Line_GGRF_TOTAL", "ExclCTdata", "Tract", "ZIP", "Census_Tract", "PP_boundary_Area"]

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist, fields_to_remove)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


## Rename a few fields

In [45]:
# Rename fields
arcpy.AlterField_management(Funding_Dist, "PP_boundary_Designatio", new_field_name="Designation")
arcpy.AlterField_management(Funding_Dist, "PP_boundary_Name", new_field_name="Tribal_Name")
arcpy.AlterField_management(Funding_Dist, "PP_boundary_Geography", new_field_name="Geography")
arcpy.AlterField_management(Funding_Dist, "PP_boundary_Tract", new_field_name="Census_Tract")


<Result 'C:\\Users\\gcowan\\Documents\\ArcGIS\\Projects\\Distribution Analysis\\Data Restructure\\Funding_Distribution_AR25\\Funding_Distribution_AR25.gdb\\Funding_Dist_AR25'>

## Create 'Wide' Data Layer

### Calculate fields for 'wide' layer


#### Percent BIPOC

In [46]:
# Percent BIPOC (PctBIPOC)

BIPOC_fields = ["Hispanic", "African_American", "Native_American", "Other_Multiple", "Asian_American", "Pacific_Islander"]
new_field = "PctBIPOC"

# Add the new field
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "DOUBLE")

# Use an UpdateCursor to calculate the sum of BIPOC percentages
with arcpy.da.UpdateCursor(Funding_Dist, BIPOC_fields + [new_field]) as cursor:
    for row in cursor:
        # Check for invalid values (None, -999, -1998) 
        if any(value in (None, -999, -1998) for value in row[:-1]):
            row[-1] = None  # Assign NULL if invalid values are present
        else:
            # Sum the values and cap at 100 if no invalid values
            total = sum(value for value in row[:-1])
            row[-1] = min(total, 100)
        cursor.updateRow(row)

print(f"Field '{new_field}' has been calculated with NULLs for invalid values.")


Field 'PctBIPOC' has been calculated with NULLs for invalid values.


#### BIPOC Majority

In [47]:
# Areas with BIOPOC Majority (BIPOCMaj)

classification_field = "PctBIPOC"
new_field = "BIPOCMaj"

# Add the new field
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "TEXT", field_length=10)

# Use an UpdateCursor to classify the rows
with arcpy.da.UpdateCursor(Funding_Dist, [classification_field, new_field]) as cursor:
    for row in cursor:
        if row[0] == 0 or row[0] is None:  # Set to NULL if PctBIPOC is 0 or null
            row[1] = None
        else:
            row[1] = "BIPOC" if row[0] > 0 else "White"
        cursor.updateRow(row)

print(f"Field '{new_field}' has been populated with NULL added for missing demographic information.")

Field 'BIPOCMaj' has been populated with NULL added for missing demographic information.


#### Demographic Majority

In [48]:
# Demographic Majoirty of Area (DemogMaj)

fields_to_compare = ["Hispanic", "African_American", "Native_American", "Other_Multiple", "Asian_American", "Pacific_Islander", "White"]
classification_field = "DemogMaj"

# Add the classification field
if classification_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, classification_field, "TEXT", field_length=50)

# Use an UpdateCursor to classify based on the greatest value
with arcpy.da.UpdateCursor(Funding_Dist, fields_to_compare + [classification_field]) as cursor:
    for row in cursor:
        # Create a dictionary of field names and their values
        field_values = {field: value for field, value in zip(fields_to_compare, row[:-1]) if value is not None}
        if field_values:  # Ensure there are non-null values
            # Find the field with the greatest value
            majority_field = max(field_values, key=field_values.get)
            row[-1] = majority_field  # Set the classification to the field name
        else:
            row[-1] = None  # Set to None if all values are null
        cursor.updateRow(row)

print(f"Field '{classification_field}' has been populated with added NULL for missing demographic information.")

Field 'DemogMaj' has been populated with added NULL for missing demographic information.


#### Total GGRF funding

In [49]:
# Get list of all program funding input fields

fields = [f.name for f in arcpy.ListFields(Funding_Dist)]

# Define keywords to search for
keywords = ["point_", "CT_", "Poly_", "Line_"]

# Use pandas to filter matching fields (case-insensitive)
df = pd.Series(fields)
matching_fields = df[df.str.contains('|'.join(keywords), case=False)].tolist()

# Print results
print("Matching fields:", matching_fields)

Matching fields: ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 'CT_PPGGRF_128', 'CT_PPGGRF_189', 'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837', 'Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78', 'Poly_NOPPGGRF_86', 'Poly_NOPPGGRF_251', 'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549', 'Poly_NOPPGGRF_695', 'Poly_PPGGRF_7', 'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 'Poly_PPGGRF_265', 'Poly_PPGGRF_695', 'Point_NOPPGGRF_1', 'Point_NOPPGGRF_2', 'Point_NOPPGGRF_3', 'Point_NOPPGGRF_5', 'Point_NOPPGGRF_11', 'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41', 'Point_NOPPGGRF_45', 'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 'Point_NOPPGGRF_76', 'Point_NOPPGGRF_78', 'Point_NOPPGGRF_86', 'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132', 'Point_NOPPGGRF_134

In [50]:
# Total GGRF funding in area (GGRF) 
    # Update field list with output from above field
GGRF_fields = ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 
               'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 
               'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 'CT_PPGGRF_128', 'CT_PPGGRF_189', 
               'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837', 'Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78', 'Poly_NOPPGGRF_86', 
               'Poly_NOPPGGRF_251', 'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549', 'Poly_NOPPGGRF_695', 
               'Poly_PPGGRF_7', 'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 'Poly_PPGGRF_265', 'Poly_PPGGRF_695',
               'Point_NOPPGGRF_1', 'Point_NOPPGGRF_2', 'Point_NOPPGGRF_3', 'Point_NOPPGGRF_5', 'Point_NOPPGGRF_11', 
               'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41', 'Point_NOPPGGRF_45', 'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 
               'Point_NOPPGGRF_76', 'Point_NOPPGGRF_78', 'Point_NOPPGGRF_86', 'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132',
               'Point_NOPPGGRF_134', 'Point_NOPPGGRF_136', 'Point_NOPPGGRF_138', 'Point_NOPPGGRF_142', 'Point_NOPPGGRF_148', 
               'Point_NOPPGGRF_191', 'Point_NOPPGGRF_193', 'Point_NOPPGGRF_235', 'Point_NOPPGGRF_237', 'Point_NOPPGGRF_239', 
               'Point_NOPPGGRF_243', 'Point_NOPPGGRF_245', 'Point_NOPPGGRF_247', 'Point_NOPPGGRF_249', 'Point_NOPPGGRF_251', 
               'Point_NOPPGGRF_253', 'Point_NOPPGGRF_255', 'Point_NOPPGGRF_257', 'Point_NOPPGGRF_259', 'Point_NOPPGGRF_261', 
               'Point_NOPPGGRF_263', 'Point_NOPPGGRF_265', 'Point_NOPPGGRF_331', 'Point_NOPPGGRF_333', 'Point_NOPPGGRF_335', 
               'Point_NOPPGGRF_339', 'Point_NOPPGGRF_341', 'Point_NOPPGGRF_345', 'Point_NOPPGGRF_471', 'Point_NOPPGGRF_475', 
               'Point_NOPPGGRF_477', 'Point_NOPPGGRF_479', 'Point_NOPPGGRF_547', 'Point_NOPPGGRF_549', 'Point_NOPPGGRF_613',
               'Point_NOPPGGRF_615', 'Point_NOPPGGRF_617', 'Point_NOPPGGRF_619', 'Point_NOPPGGRF_621', 'Point_NOPPGGRF_690',
               'Point_NOPPGGRF_765', 'Point_NOPPGGRF_839', 'Point_NOPPGGRF_981', 'Point_NOPPGGRF_1140', 'Point_NOPPGGRF_1144',
               'Point_NOPPGGRF_1291', 'Point_NOPPGGRF_1309', 'Point_NOPPGGRF_1327', 'Point_NOPPGGRF_1399', 'Point_NOPPGGRF_1417',
               'Point_NOPPGGRF_1435', 'Point_PPGGRF_1', 'Point_PPGGRF_3', 'Point_PPGGRF_5', 'Point_PPGGRF_7', 'Point_PPGGRF_10', 
               'Point_PPGGRF_11', 'Point_PPGGRF_14', 'Point_PPGGRF_41', 'Point_PPGGRF_45', 'Point_PPGGRF_68', 'Point_PPGGRF_76',
               'Point_PPGGRF_78', 'Point_PPGGRF_86', 'Point_PPGGRF_120', 'Point_PPGGRF_124', 'Point_PPGGRF_130', 
               'Point_PPGGRF_132', 'Point_PPGGRF_134', 'Point_PPGGRF_136', 'Point_PPGGRF_138', 'Point_PPGGRF_142', 
               'Point_PPGGRF_148', 'Point_PPGGRF_191', 'Point_PPGGRF_193', 'Point_PPGGRF_235', 'Point_PPGGRF_237', 
               'Point_PPGGRF_239', 'Point_PPGGRF_245', 'Point_PPGGRF_247', 'Point_PPGGRF_251', 'Point_PPGGRF_253', 
               'Point_PPGGRF_257', 'Point_PPGGRF_259', 'Point_PPGGRF_261', 'Point_PPGGRF_263', 'Point_PPGGRF_265', 
               'Point_PPGGRF_267', 'Point_PPGGRF_269', 'Point_PPGGRF_271', 'Point_PPGGRF_333', 'Point_PPGGRF_335', 
               'Point_PPGGRF_339', 'Point_PPGGRF_341', 'Point_PPGGRF_345', 'Point_PPGGRF_471', 'Point_PPGGRF_475', 
               'Point_PPGGRF_479', 'Point_PPGGRF_547', 'Point_PPGGRF_549', 'Point_PPGGRF_613', 'Point_PPGGRF_619', 
               'Point_PPGGRF_621', 'Point_PPGGRF_690', 'Point_PPGGRF_695', 'Point_PPGGRF_765', 'Point_PPGGRF_1291', 
               'Point_PPGGRF_1309', 'Point_PPGGRF_1327', 'Point_PPGGRF_1417', 'Point_PPGGRF_1435', 'Line_NOPPGGRF_68',
               'Line_NOPPGGRF_78', 'Line_NOPPGGRF_549', 'Line_PPGGRF_68', 'Line_PPGGRF_78']

new_field = "GGRF"

# Add the new field 
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "DOUBLE")

# Use an UpdateCursor to calculate the sum of the fields
with arcpy.da.UpdateCursor(Funding_Dist, GGRF_fields + [new_field]) as cursor:
    for row in cursor:
        # Sum the values of the fields, treating None as 0
        total = sum(value if value is not None else 0 for value in row[:-1])  
        row[-1] = total  # Set the sum in the new field
        cursor.updateRow(row)

print(f"Field '{new_field}' has been calculated.")

Field 'GGRF' has been calculated.


In [51]:
# Check GGRF field total

GGRF_sum = 0

# Use a SearchCursor to iterate through the rows
with arcpy.da.SearchCursor(Funding_Dist, ['GGRF']) as cursor:
    for row in cursor:
        # Add the value of the field to the total sum
        GGRF_sum += row[0]

# Print the result
print(f"The total sum of the field '{'GGRF_sum'}' is: {GGRF_sum}")

The total sum of the field 'GGRF_sum' is: 12775208202.749277


#### Percentile ranking of all areas based on GGRF funding

In [52]:
# Percentile ranking of each area based on total GGRF funding within its respective geography (CCI_pct) 

input_field = "GGRF"
percentile_field = "CCI_pct"

# Add the new percentile field 
if percentile_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, percentile_field, "DOUBLE")

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Calculate percentiles as decimals
values = sorted(values)  # Sort values for percentile calculation
percentile_ranks = {value: i / (len(values) - 1) for i, value in enumerate(values)} 

# Step 3: Update the percentile field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, percentile_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values
            # Assign the rounded decimal percentile rank to the new field
            row[1] = round(percentile_ranks[row[0]], 2)
        else:
            row[1] = None  # Assign NULL for invalid values
        cursor.updateRow(row)

print(f"CCI Percentile values rounded to two decimal places have been calculated and stored in the field '{percentile_field}'.")


CCI Percentile values rounded to two decimal places have been calculated and stored in the field 'CCI_pct'.


#### Decile ranking of all areas based on GGRF funding

In [53]:
# Decile ranking of each area based on total GGRF funding within its respective geography (CCIdecile)

input_field = "GGRF"
category_field = "CCIdecile" 

# Add the new category field 
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories
values = sorted(values)  # Sort values for decile calculation
categories = [
    "Lowest 10 Percentiles",
    ">10th-20th Percentile",
    ">20th-30th Percentile",
    ">30th-40th Percentile",
    ">40th-50th Percentile",
    ">50th-60th Percentile",
    ">60th-70th Percentile",
    ">70th-80th Percentile",
    ">80th-90th Percentile",
    "Highest 10 Percentiles"
]
category_mapping = {value: categories[int(i / (len(values) / 10))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values
            # Assign the category label to the new field
            row[1] = category_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL for invalid values
        cursor.updateRow(row)

print(f"CCI deciles have been calculated and stored in the field '{category_field}'.")


CCI deciles have been calculated and stored in the field 'CCIdecile'.


#### Quartile ranking of all areas based on GGRF funding

In [54]:
# Quartile ranking of each area based on total GGRF funding within its respective geography (CCIquartile) 

input_field = "GGRF"
category_field = "CCIquartile" 

# Add the new category field 
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories
values = sorted(values)  # Sort values for quartile calculation
categories = [
    "<25th Percentile",       # 0% to 25%
    ">25th-50th Percentile",   # 25% to 50%
    ">50th-75th Percentile",   # 50% to 75%
    ">75th Percentile"       # 75% to 100%
]
quartile_mapping = {value: categories[int(i / (len(values) / 4))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values
            # Assign the quartile label to the new field
            row[1] = quartile_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL for invalid values
        cursor.updateRow(row)

print(f"CCI quartile categories have been calculated and stored in the field '{category_field}'.")


CCI quartile categories have been calculated and stored in the field 'CCIquartile'.


#### GGRF funding per capita

In [55]:
# GGRF funding per capita for each area (CCIpc)

input_field = "GGRF"
population_field = "Population"
per_capita_field = "CCIpc"

# Add the new per capita field 
if per_capita_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, per_capita_field, "DOUBLE") 

# Calculate per capita funding
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, population_field, per_capita_field]) as cursor:
    for row in cursor:
        funding = row[0]
        population = row[1]

        # Check if population is valid and non-zero
        if population and population > 0:
            row[2] = funding / population  # Calculate per capita funding
        else:
            row[2] = None  # Assign NULL if population is 0 or None

        cursor.updateRow(row)

print(f"Per capita funding values have been calculated and stored in the field '{per_capita_field}'.")


Per capita funding values have been calculated and stored in the field 'CCIpc'.


#### Percentile ranking of areas based on per capita funding

In [56]:
# GGRF funding per capita percentile ranking (CCIpcPct)

input_field = "CCIpc"
percentile_field = "CCIpcPct"

# Add the new percentile field
if percentile_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, percentile_field, "DOUBLE")

# Step 1: Extract values from the source field (ignoring None values)
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Ensure we have at least one valid value
if len(values) > 0:
    # Step 2: Calculate percentiles as decimals
    values = sorted(values)  # Sort values for percentile calculation
    percentile_ranks = {value: i / (len(values) - 1) for i, value in enumerate(values)}  # Decimal percentile rank

    # Step 3: Update the percentile field
    with arcpy.da.UpdateCursor(Funding_Dist, [input_field, percentile_field]) as cursor:
        for row in cursor:
            if row[0] is not None:  # Check for valid values in CCIpc
                # Assign the percentile rank as a decimal, rounded to two decimal places
                row[1] = round(percentile_ranks[row[0]], 2)
            else:
                row[1] = None  # Assign NULL for invalid values (None in CCIpc)
            cursor.updateRow(row)

    print(f"CCIpc Percentile values rounded to two decimal places have been calculated and stored in the field '{percentile_field}'.")
else:
    print("No valid values found in the input field.")


CCIpc Percentile values rounded to two decimal places have been calculated and stored in the field 'CCIpcPct'.


#### Decile ranking of areas based on per capita funding

In [57]:
# GGRF funding per capita decile ranking (CCIpcDecile)

input_field = "CCIpc"
category_field = "CCIpcDecile" 

# Add the new category field
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50)

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories
values = sorted(values)  # Sort values for decile calculation
categories = [
    "Lowest 10 Percentiles",
    ">10th-20th Percentile",
    ">20th-30th Percentile",
    ">30th-40th Percentile",
    ">40th-50th Percentile",
    ">50th-60th Percentile",
    ">60th-70th Percentile",
    ">70th-80th Percentile",
    ">80th-90th Percentile",
    "Highest 10 Percentiles"
]

# Calculate deciles and map to categories
category_mapping = {value: categories[int(i / (len(values) / 10))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CCIpc
            # Assign the appropriate category label
            row[1] = category_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CCIpc is None
        cursor.updateRow(row)

print(f"CCI categories have been calculated and stored in the field '{category_field}'.")


CCI categories have been calculated and stored in the field 'CCIpcDecile'.


#### Quartile ranking of areas based on per capita funding

In [58]:
# GGRF funding per capita quartile ranking (CCIpcQuartile)

input_field = "CCIpc"
category_field = "CCIpcQuartile"  

# Add the new category field if it doesn't exist
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories for quartiles
values = sorted(values)  # Sort values for quartile calculation
categories = [
    "<25th Percentile",       # 0% to 25%
    ">25th-50th Percentile",   # 25% to 50%
    ">50th-75th Percentile",   # 50% to 75%
    ">75th Percentile"       # 75% to 100%
]

# Calculate quartiles and map to categories
quartile_mapping = {value: categories[int(i / (len(values) / 4))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CCIpc
            # Assign the appropriate quartile label
            row[1] = quartile_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CCIpc is None
        cursor.updateRow(row)

print(f"CCI quartile categories have been calculated and stored in the field '{category_field}'.")


CCI quartile categories have been calculated and stored in the field 'CCIpcQuartile'.


#### 5% ranges of CES score

In [59]:
# Groups areas based on 5% increments based on CES 4.0 score (CES4range)

input_field = "CIscoreP"
range_field = "CES4range" 

# Add the new range field
if range_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, range_field, "TEXT", field_length=50)

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define 5% range categories
values = sorted(values)  # Sort values
categories = [f"{i * 5}%-{(i + 1) * 5}%" for i in range(20)]  # 5% ranges from 0% to 100%

# Map values to 5% ranges
range_mapping = {}
bin_size = len(values) / 20  # Divide sorted values into 20 bins (5% each)
for i, value in enumerate(values):
    range_mapping[value] = categories[int(i // bin_size)]  # Assign range category

# Step 3: Update the range field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, range_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CIscoreP
            # Assign the appropriate range label
            row[1] = range_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CIscoreP is None
        cursor.updateRow(row)

print(f"5% range categories have been calculated and stored in the field '{range_field}'.")


5% range categories have been calculated and stored in the field 'CES4range'.


#### 10% ranges based on CES score

In [60]:
# Groups areas based on 10% increments based on CES 4.0 score (CES4decile)

input_field = "CIscoreP"
decile_field = "CES4decile"  

# Add the new decile field 
if decile_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, decile_field, "TEXT", field_length=50)  

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define 10% range categories
values = sorted(values)  # Sort values
categories = [f"{i * 10}%-{(i + 1) * 10}%" for i in range(10)]  # 10% ranges from 0% to 100%

# Map values to 10% ranges
range_mapping = {}
bin_size = len(values) / 10  # Divide sorted values into 10 bins (10% each)
for i, value in enumerate(values):
    range_mapping[value] = categories[int(i // bin_size)]  # Assign range category

# Step 3: Update the decile field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, decile_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CIscoreP
            # Assign the appropriate decile label
            row[1] = range_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CIscoreP is None
        cursor.updateRow(row)

print(f"10% range categories have been calculated and stored in the field '{decile_field}'.")


10% range categories have been calculated and stored in the field 'CES4decile'.


#### Simplified priority population designations

In [61]:
# Simplified priority population designation based on the "Designation" field (PP_simple)

input_field = "Designation"  
output_field = "PP_simple" 

# Add the new field 
if output_field not in [field.name for field in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, output_field, "TEXT", field_length=100)

# Define the aggregation logic as a dictionary
category_mapping = {
    "DAC 1/2 mile neighbor: low-income community": "DAC 1/2 Mile Neighbor: Low-Income Community",
    "Low-income community": "Low-Income Community",
    "Disadvantaged community: CES": "Disadvantaged Community",
    "Disadvantaged community: CES, Disadvantaged community: Tribal land, Low-income community": "Disadvantaged Community",
    "Disadvantaged community: CES, Low-income community": "Disadvantaged Community",
    "Disadvantaged community: Tribal land": "Disadvantaged Community",
    "Disadvantaged community: Tribal land, Low-income community": "Disadvantaged Community",
    "DAC 1/2 mile neighbor: low-income household eligible": "Not a Priority Population Area: Low-Income Households are Eligible",
    "Low-income household eligible": "Not a Priority Population Area: Low-Income Households are Eligible"
}

# Update the new field based on the input field's values
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, output_field]) as cursor:
    for row in cursor:
        input_value = row[0]  # Value from the input field
        # Determine the category based on the mapping
        category = category_mapping.get(input_value, "Unknown")  # Default to "Unknown" if no match
        row[1] = category  # Set the category in the output field
        cursor.updateRow(row)

print("Simplified priority population categorization completed successfully.")


Simplified priority population categorization completed successfully.


### Create program sum fields

In [62]:
# generate and format a list of all funding columns

# List all field names
fields = [f.name for f in arcpy.ListFields(Funding_Dist)]

# Define keywords to search for
keywords = ["point_", "CT_", "Poly_", "Line_"]

# Filter fields with matching keywords
df = pd.Series(fields)
matching_fields = df[df.str.contains('|'.join(keywords), case=False)].tolist()

# Group fields by numeric suffix only
grouped_dict = {}
for field in matching_fields:
    if "_" in field:
        suffix2 = field.rsplit("_", 1)[-1]
        if suffix2.isdigit():  # Only include numeric suffixes
            grouped_dict.setdefault(suffix2, []).append(field)

# Print in desired format
print("{")
for key in sorted(grouped_dict, key=lambda x: int(x)):
    values = grouped_dict[key]
    formatted_values = ", ".join(f'"{v}"' for v in values)
    print(f'    "{key}": [{formatted_values}],')
print("}")

{
    "1": ["Point_NOPPGGRF_1", "Point_PPGGRF_1"],
    "2": ["Point_NOPPGGRF_2"],
    "3": ["Point_NOPPGGRF_3", "Point_PPGGRF_3"],
    "4": ["CT_NOPPGGRF_4", "CT_PPGGRF_4"],
    "5": ["Point_NOPPGGRF_5", "Point_PPGGRF_5"],
    "7": ["Poly_PPGGRF_7", "Point_PPGGRF_7"],
    "8": ["CT_NOPPGGRF_8", "CT_PPGGRF_8"],
    "9": ["CT_NOPPGGRF_9", "CT_PPGGRF_9"],
    "10": ["Point_PPGGRF_10"],
    "11": ["Point_NOPPGGRF_11", "Point_PPGGRF_11"],
    "14": ["Point_NOPPGGRF_14", "Point_PPGGRF_14"],
    "41": ["Point_NOPPGGRF_41", "Point_PPGGRF_41"],
    "45": ["Point_NOPPGGRF_45", "Point_PPGGRF_45"],
    "68": ["Poly_NOPPGGRF_68", "Poly_PPGGRF_68", "Point_NOPPGGRF_68", "Point_PPGGRF_68", "Line_NOPPGGRF_68", "Line_PPGGRF_68"],
    "72": ["Point_NOPPGGRF_72"],
    "76": ["Point_NOPPGGRF_76", "Point_PPGGRF_76"],
    "78": ["Poly_NOPPGGRF_78", "Poly_PPGGRF_78", "Point_NOPPGGRF_78", "Point_PPGGRF_78", "Line_NOPPGGRF_78", "Line_PPGGRF_78"],
    "80": ["CT_NOPPGGRF_80", "CT_PPGGRF_80"],
    "82": ["CT_NOPP

In [63]:
# Create program specific funding columns 
    # paste the list from the above output into the field_groups section of the code 
    # combine the fields needed from subprogram names list

Funding_Dist = f"{project_gdb}\\Funding_Dist_{suffix}"

# Define groups based on suffixes
field_groups = {
    "1": ["Point_NOPPGGRF_1", "Point_PPGGRF_1"],
    "2": ["Point_NOPPGGRF_2"],
    "3": ["Point_NOPPGGRF_3", "Point_PPGGRF_3"],
    "4": ["CT_NOPPGGRF_4", "CT_PPGGRF_4"],
    "5": ["Point_NOPPGGRF_5", "Point_PPGGRF_5"],
    "7": ["Poly_PPGGRF_7", "Point_PPGGRF_7"],
    "8": ["CT_NOPPGGRF_8", "CT_PPGGRF_8"],
    "9": ["CT_NOPPGGRF_9", "CT_PPGGRF_9"],
    "10": ["Point_PPGGRF_10", "Point_PPGGRF_269", "Point_PPGGRF_271", "Point_NOPPGGRF_547", "Point_PPGGRF_547"],
    "11": ["Point_NOPPGGRF_11", "Point_PPGGRF_11"],
    "14": ["Point_NOPPGGRF_14", "Point_PPGGRF_14"],
    "41": ["Point_NOPPGGRF_41", "Point_PPGGRF_41"],
    "45": ["Point_NOPPGGRF_45", "Point_PPGGRF_45"],
    "68": ["Poly_NOPPGGRF_68", "Poly_PPGGRF_68", "Point_NOPPGGRF_68", "Point_PPGGRF_68", "Line_NOPPGGRF_68", "Line_PPGGRF_68"],
    "72": ["Point_NOPPGGRF_72"],
    "76": ["Point_NOPPGGRF_76", "Point_PPGGRF_76"],
    "78": ["Poly_NOPPGGRF_78", "Poly_PPGGRF_78", "Point_NOPPGGRF_78", "Point_PPGGRF_78", "Line_NOPPGGRF_78", "Line_PPGGRF_78"],
    "80": ["CT_NOPPGGRF_80", "CT_PPGGRF_80"],
    "82": ["CT_NOPPGGRF_82", "CT_PPGGRF_82"],
    "86": ["Poly_NOPPGGRF_86", "Point_NOPPGGRF_86", "Point_PPGGRF_86"],
    "88": ["CT_NOPPGGRF_88", "CT_PPGGRF_88"],
    "90": ["CT_NOPPGGRF_90", "CT_PPGGRF_90"],
    "120": ["Point_PPGGRF_120"],
    "124": ["Point_NOPPGGRF_124", "Point_PPGGRF_124"],
    "128": ["CT_NOPPGGRF_128", "CT_PPGGRF_128"],
    "130": ["Point_PPGGRF_130"],
    "132": ["Point_NOPPGGRF_132", "Point_PPGGRF_132"],
    "134": ["Point_NOPPGGRF_134", "Point_PPGGRF_134"],
    "136": ["Point_NOPPGGRF_136", "Point_PPGGRF_136"],
    "138": ["Point_NOPPGGRF_138", "Point_PPGGRF_138"],
    "142": ["Point_NOPPGGRF_142", "Point_PPGGRF_142", "CT_PPGGRF_329"],
    "148": ["Point_NOPPGGRF_148", "Point_PPGGRF_148"],
    "189": ["CT_PPGGRF_189"],
    "191": ["Point_NOPPGGRF_191", "Point_PPGGRF_191"],
    "193": ["Point_NOPPGGRF_193", "Point_PPGGRF_193"],
    "235": ["Point_NOPPGGRF_235", "Point_PPGGRF_235"],
    "237": ["Point_NOPPGGRF_237", "Point_PPGGRF_237"],
    "239": ["Point_NOPPGGRF_239", "Point_PPGGRF_239"],
    "243": ["Point_NOPPGGRF_243"],
    "245": ["Point_NOPPGGRF_245", "Point_PPGGRF_245"],
    "247": ["Point_NOPPGGRF_247", "Point_PPGGRF_247"],
    "249": ["Point_NOPPGGRF_249"],
    "251": ["Poly_NOPPGGRF_251", "Poly_PPGGRF_251", "Point_NOPPGGRF_251", "Point_PPGGRF_251"],
    "253": ["Point_NOPPGGRF_253", "Point_PPGGRF_253"],
    "255": ["Point_NOPPGGRF_255"],
    "257": ["Point_NOPPGGRF_257", "Point_PPGGRF_257"],
    "259": ["Point_NOPPGGRF_259", "Point_PPGGRF_259"],
    "261": ["Point_NOPPGGRF_261", "Point_PPGGRF_261"],
    "263": ["Point_NOPPGGRF_263", "Point_PPGGRF_263"],
    "265": ["Poly_NOPPGGRF_265", "Poly_PPGGRF_265", "Point_NOPPGGRF_265", "Point_PPGGRF_265"],
    "267": ["Point_PPGGRF_267"],
    "331": ["Point_NOPPGGRF_331"],
    "333": ["Point_NOPPGGRF_333", "Point_PPGGRF_333"],
    "335": ["Point_NOPPGGRF_335", "Point_PPGGRF_335"],
    "337": ["CT_NOPPGGRF_337", "CT_PPGGRF_337"],
    "339": ["Point_NOPPGGRF_339", "Point_PPGGRF_339"],
    "341": ["Point_NOPPGGRF_341", "Point_PPGGRF_341"],
    "345": ["Poly_NOPPGGRF_345", "Point_NOPPGGRF_345", "Point_PPGGRF_345"],
    "471": ["Point_NOPPGGRF_471", "Point_PPGGRF_471"],
    "475": ["Point_NOPPGGRF_475", "Point_PPGGRF_475"],
    "477": ["Point_NOPPGGRF_477"],
    "479": ["Point_NOPPGGRF_479", "Point_PPGGRF_479"],
    "549": ["Poly_NOPPGGRF_549", "Point_NOPPGGRF_549", "Point_PPGGRF_549", "Line_NOPPGGRF_549"],
    "613": ["Point_NOPPGGRF_613", "Point_PPGGRF_613"],
    "615": ["Point_NOPPGGRF_615"],
    "617": ["Point_NOPPGGRF_617"],
    "619": ["Point_NOPPGGRF_619", "Point_PPGGRF_619"],
    "621": ["Point_NOPPGGRF_621", "Point_PPGGRF_621"],
    "690": ["Point_NOPPGGRF_690", "Point_PPGGRF_690"],
    "695": ["Poly_NOPPGGRF_695", "Poly_PPGGRF_695", "Point_PPGGRF_695"],
    "765": ["Point_NOPPGGRF_765", "Point_PPGGRF_765"],
    "837": ["CT_PPGGRF_837"],
    "839": ["Point_NOPPGGRF_839"],
    "981": ["Point_NOPPGGRF_981"],
    "1136": ["CT_NOPPGGRF_1136"],
    "1140": ["Point_NOPPGGRF_1140"],
    "1144": ["Point_NOPPGGRF_1144"],
    "1291": ["Point_NOPPGGRF_1291", "Point_PPGGRF_1291"],
    "1309": ["Point_NOPPGGRF_1309", "Point_PPGGRF_1309"],
    "1327": ["Point_NOPPGGRF_1327", "Point_PPGGRF_1327"],
    "1399": ["Point_NOPPGGRF_1399"],
    "1417": ["Point_NOPPGGRF_1417", "Point_PPGGRF_1417"],
    "1435": ["Point_NOPPGGRF_1435", "Point_PPGGRF_1435"],
}

# Add a field for each group and calculate the sum of the grouped fields
for group, fields in field_groups.items():
    # Add a new field for the summed value of each group
    new_field_name = f"sum_group_{group}"
    arcpy.AddField_management(Funding_Dist, new_field_name, "DOUBLE")
    
    # Use an update cursor to calculate the sum for each record
    with arcpy.da.UpdateCursor(Funding_Dist, fields + [new_field_name]) as cursor:
        for row in cursor:
            # Sum the values of the fields in the group
            total_sum = sum([row[fields.index(field)] if row[fields.index(field)] is not None else 0 for field in fields])
            row[-1] = total_sum  # Assign the sum to the new field
            cursor.updateRow(row)

print("Fields have been grouped and summed successfully.")

Fields have been grouped and summed successfully.


### Check totals prior to saving layer

In [74]:
# List all field names
fields = [f.name for f in arcpy.ListFields(Funding_Dist)]

# Filter fields that start with 'sum_group_'
sum_group_fields = [field for field in fields if field.startswith("sum_group_")]

# Print the result
print("Fields that start with 'sum_group_':")
print(sum_group_fields)

Fields that start with 'sum_group_':
['sum_group_1', 'sum_group_2', 'sum_group_3', 'sum_group_4', 'sum_group_5', 'sum_group_7', 'sum_group_8', 'sum_group_9', 'sum_group_10', 'sum_group_11', 'sum_group_14', 'sum_group_41', 'sum_group_45', 'sum_group_68', 'sum_group_72', 'sum_group_76', 'sum_group_78', 'sum_group_80', 'sum_group_82', 'sum_group_86', 'sum_group_88', 'sum_group_90', 'sum_group_120', 'sum_group_124', 'sum_group_128', 'sum_group_130', 'sum_group_132', 'sum_group_134', 'sum_group_136', 'sum_group_138', 'sum_group_142', 'sum_group_148', 'sum_group_189', 'sum_group_191', 'sum_group_193', 'sum_group_235', 'sum_group_237', 'sum_group_239', 'sum_group_243', 'sum_group_245', 'sum_group_247', 'sum_group_249', 'sum_group_251', 'sum_group_253', 'sum_group_255', 'sum_group_257', 'sum_group_259', 'sum_group_261', 'sum_group_263', 'sum_group_265', 'sum_group_267', 'sum_group_331', 'sum_group_333', 'sum_group_335', 'sum_group_337', 'sum_group_339', 'sum_group_341', 'sum_group_345', 'sum_g

In [75]:
# Check sum of all program funding from calculated fields
    # Copy the list from above into the program_sum_fields list

program_sum_fields = ['sum_group_1', 'sum_group_2', 'sum_group_3', 'sum_group_4', 'sum_group_5', 'sum_group_7', 'sum_group_8', 'sum_group_9', 'sum_group_10', 
                      'sum_group_11', 'sum_group_14', 'sum_group_41', 'sum_group_45', 'sum_group_68', 'sum_group_72', 'sum_group_76', 'sum_group_78', 
                      'sum_group_80', 'sum_group_82', 'sum_group_86', 'sum_group_88', 'sum_group_90', 'sum_group_120', 'sum_group_124', 'sum_group_128', 
                      'sum_group_130', 'sum_group_132', 'sum_group_134', 'sum_group_136', 'sum_group_138', 'sum_group_142', 'sum_group_148', 'sum_group_189', 
                      'sum_group_191', 'sum_group_193', 'sum_group_235', 'sum_group_237', 'sum_group_239', 'sum_group_243', 'sum_group_245', 'sum_group_247', 
                      'sum_group_249', 'sum_group_251', 'sum_group_253', 'sum_group_255', 'sum_group_257', 'sum_group_259', 'sum_group_261', 'sum_group_263', 
                      'sum_group_265', 'sum_group_267', 'sum_group_331', 'sum_group_333', 'sum_group_335', 'sum_group_337', 'sum_group_339', 'sum_group_341', 
                      'sum_group_345', 'sum_group_471', 'sum_group_475', 'sum_group_477', 'sum_group_479', 'sum_group_549', 'sum_group_613', 'sum_group_615', 
                      'sum_group_617', 'sum_group_619', 'sum_group_621', 'sum_group_690', 'sum_group_695', 'sum_group_765', 'sum_group_837', 'sum_group_839', 
                      'sum_group_981', 'sum_group_1136', 'sum_group_1140', 'sum_group_1144', 'sum_group_1291', 'sum_group_1309', 'sum_group_1327', 
                      'sum_group_1399', 'sum_group_1417', 'sum_group_1435']


# Initialize a total variable for all fields
total_sum = 0

# Use a SearchCursor to iterate through the field values
with arcpy.da.SearchCursor(Funding_Dist, program_sum_fields) as cursor:
    for row in cursor:
        for value in row:
            if value is not None:  # Ensure the value is not NULL
                total_sum += value

# Round and print the result
total_sum = round(total_sum)
print(f"The total sum of all summed calculated program fields is: {total_sum:,}")

The total sum of all summed calculated program fields is: 12,775,208,203


In [76]:
# Get list of all program funding input fields

fields = [f.name for f in arcpy.ListFields(Funding_Dist)]

# Define keywords to search for
keywords = ["point_", "CT_", "Poly_", "Line_"]

# Use pandas to filter matching fields (case-insensitive)
df = pd.Series(fields)
matching_fields = df[df.str.contains('|'.join(keywords), case=False)].tolist()

# Print results
print("Matching fields:", matching_fields)

Matching fields: ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 'CT_PPGGRF_128', 'CT_PPGGRF_189', 'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837', 'Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78', 'Poly_NOPPGGRF_86', 'Poly_NOPPGGRF_251', 'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549', 'Poly_NOPPGGRF_695', 'Poly_PPGGRF_7', 'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 'Poly_PPGGRF_265', 'Poly_PPGGRF_695', 'Point_NOPPGGRF_1', 'Point_NOPPGGRF_2', 'Point_NOPPGGRF_3', 'Point_NOPPGGRF_5', 'Point_NOPPGGRF_11', 'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41', 'Point_NOPPGGRF_45', 'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 'Point_NOPPGGRF_76', 'Point_NOPPGGRF_78', 'Point_NOPPGGRF_86', 'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132', 'Point_NOPPGGRF_134

In [77]:
# Check sum of all program funding from the input data fields (to check against the total from calculated fields above)
    # Paste the list above into the "fields" variable below and remove the "total" fields
    
fields = ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 
          'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 
          'CT_PPGGRF_128', 'CT_PPGGRF_189', 'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837', 
          'Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78', 'Poly_NOPPGGRF_86', 'Poly_NOPPGGRF_251', 'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549', 
          'Poly_NOPPGGRF_695', 'Poly_PPGGRF_7', 'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 'Poly_PPGGRF_265', 'Poly_PPGGRF_695', 
          'Point_NOPPGGRF_1', 'Point_NOPPGGRF_2', 'Point_NOPPGGRF_3', 'Point_NOPPGGRF_5', 
          'Point_NOPPGGRF_11', 'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41', 'Point_NOPPGGRF_45', 'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 'Point_NOPPGGRF_76', 
          'Point_NOPPGGRF_78', 'Point_NOPPGGRF_86', 'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132', 'Point_NOPPGGRF_134', 'Point_NOPPGGRF_136', 
          'Point_NOPPGGRF_138', 'Point_NOPPGGRF_142', 'Point_NOPPGGRF_148', 'Point_NOPPGGRF_191', 'Point_NOPPGGRF_193', 'Point_NOPPGGRF_235', 
          'Point_NOPPGGRF_237', 'Point_NOPPGGRF_239', 'Point_NOPPGGRF_243', 'Point_NOPPGGRF_245', 'Point_NOPPGGRF_247', 'Point_NOPPGGRF_249', 
          'Point_NOPPGGRF_251', 'Point_NOPPGGRF_253', 'Point_NOPPGGRF_255', 'Point_NOPPGGRF_257', 'Point_NOPPGGRF_259', 'Point_NOPPGGRF_261', 
          'Point_NOPPGGRF_263', 'Point_NOPPGGRF_265', 'Point_NOPPGGRF_331', 'Point_NOPPGGRF_333', 'Point_NOPPGGRF_335', 'Point_NOPPGGRF_339', 
          'Point_NOPPGGRF_341', 'Point_NOPPGGRF_345', 'Point_NOPPGGRF_471', 'Point_NOPPGGRF_475', 'Point_NOPPGGRF_477', 'Point_NOPPGGRF_479', 
          'Point_NOPPGGRF_547', 'Point_NOPPGGRF_549', 'Point_NOPPGGRF_613', 'Point_NOPPGGRF_615', 'Point_NOPPGGRF_617', 'Point_NOPPGGRF_619', 
          'Point_NOPPGGRF_621', 'Point_NOPPGGRF_690', 'Point_NOPPGGRF_765', 'Point_NOPPGGRF_839', 'Point_NOPPGGRF_981', 'Point_NOPPGGRF_1140', 
          'Point_NOPPGGRF_1144', 'Point_NOPPGGRF_1291', 'Point_NOPPGGRF_1309', 'Point_NOPPGGRF_1327', 'Point_NOPPGGRF_1399', 'Point_NOPPGGRF_1417', 
          'Point_NOPPGGRF_1435', 'Point_PPGGRF_1', 'Point_PPGGRF_3', 'Point_PPGGRF_5', 'Point_PPGGRF_7', 'Point_PPGGRF_10', 'Point_PPGGRF_11', 
          'Point_PPGGRF_14', 'Point_PPGGRF_41', 'Point_PPGGRF_45', 'Point_PPGGRF_68', 'Point_PPGGRF_76', 'Point_PPGGRF_78', 'Point_PPGGRF_86', 
          'Point_PPGGRF_120', 'Point_PPGGRF_124', 'Point_PPGGRF_130', 'Point_PPGGRF_132', 'Point_PPGGRF_134', 'Point_PPGGRF_136', 'Point_PPGGRF_138', 
          'Point_PPGGRF_142', 'Point_PPGGRF_148', 'Point_PPGGRF_191', 'Point_PPGGRF_193', 'Point_PPGGRF_235', 'Point_PPGGRF_237', 'Point_PPGGRF_239', 
          'Point_PPGGRF_245', 'Point_PPGGRF_247', 'Point_PPGGRF_251', 'Point_PPGGRF_253', 'Point_PPGGRF_257', 'Point_PPGGRF_259', 'Point_PPGGRF_261',
          'Point_PPGGRF_263', 'Point_PPGGRF_265', 'Point_PPGGRF_267', 'Point_PPGGRF_269', 'Point_PPGGRF_271', 'Point_PPGGRF_333', 'Point_PPGGRF_335', 
          'Point_PPGGRF_339', 'Point_PPGGRF_341', 'Point_PPGGRF_345', 'Point_PPGGRF_471', 'Point_PPGGRF_475', 'Point_PPGGRF_479', 'Point_PPGGRF_547',
          'Point_PPGGRF_549', 'Point_PPGGRF_613', 'Point_PPGGRF_619', 'Point_PPGGRF_621', 'Point_PPGGRF_690', 'Point_PPGGRF_695', 'Point_PPGGRF_765',
          'Point_PPGGRF_1291', 'Point_PPGGRF_1309', 'Point_PPGGRF_1327', 'Point_PPGGRF_1417', 'Point_PPGGRF_1435', 
          'Line_NOPPGGRF_68', 'Line_NOPPGGRF_78', 'Line_NOPPGGRF_549', 'Line_PPGGRF_68', 'Line_PPGGRF_78']


# Initialize a total variable for all fields
total_sum = 0

# Use a SearchCursor to iterate through the field values
with arcpy.da.SearchCursor(Funding_Dist, fields) as cursor:
    for row in cursor:
        for value in row:
            if value is not None:  # Ensure the value is not NULL
                total_sum += value

# Round and print the result
total_sum = round(total_sum)
print(f"The total sum of all summed input program fields is: {total_sum:,}")

The total sum of all summed input program fields is: 12,775,208,203


In [78]:
# Calculate SUM of totals using input totals (to check against above program total) - QC

# List of fields to sum
fields_CT = ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 
          'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 
          'CT_PPGGRF_128', 'CT_PPGGRF_189', 'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837']

fields_poly = ['Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78', 'Poly_NOPPGGRF_86', 'Poly_NOPPGGRF_251', 'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549', 
          'Poly_NOPPGGRF_695', 'Poly_PPGGRF_7', 'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 'Poly_PPGGRF_265', 'Poly_PPGGRF_695']

fields_point = ['Point_NOPPGGRF_1', 'Point_NOPPGGRF_2', 'Point_NOPPGGRF_3', 'Point_NOPPGGRF_5', 
          'Point_NOPPGGRF_11', 'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41', 'Point_NOPPGGRF_45', 'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 'Point_NOPPGGRF_76', 
          'Point_NOPPGGRF_78', 'Point_NOPPGGRF_86', 'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132', 'Point_NOPPGGRF_134', 'Point_NOPPGGRF_136', 
          'Point_NOPPGGRF_138', 'Point_NOPPGGRF_142', 'Point_NOPPGGRF_148', 'Point_NOPPGGRF_191', 'Point_NOPPGGRF_193', 'Point_NOPPGGRF_235', 
          'Point_NOPPGGRF_237', 'Point_NOPPGGRF_239', 'Point_NOPPGGRF_243', 'Point_NOPPGGRF_245', 'Point_NOPPGGRF_247', 'Point_NOPPGGRF_249', 
          'Point_NOPPGGRF_251', 'Point_NOPPGGRF_253', 'Point_NOPPGGRF_255', 'Point_NOPPGGRF_257', 'Point_NOPPGGRF_259', 'Point_NOPPGGRF_261', 
          'Point_NOPPGGRF_263', 'Point_NOPPGGRF_265', 'Point_NOPPGGRF_331', 'Point_NOPPGGRF_333', 'Point_NOPPGGRF_335', 'Point_NOPPGGRF_339', 
          'Point_NOPPGGRF_341', 'Point_NOPPGGRF_345', 'Point_NOPPGGRF_471', 'Point_NOPPGGRF_475', 'Point_NOPPGGRF_477', 'Point_NOPPGGRF_479', 
          'Point_NOPPGGRF_547', 'Point_NOPPGGRF_549', 'Point_NOPPGGRF_613', 'Point_NOPPGGRF_615', 'Point_NOPPGGRF_617', 'Point_NOPPGGRF_619', 
          'Point_NOPPGGRF_621', 'Point_NOPPGGRF_690', 'Point_NOPPGGRF_765', 'Point_NOPPGGRF_839', 'Point_NOPPGGRF_981', 'Point_NOPPGGRF_1140', 
          'Point_NOPPGGRF_1144', 'Point_NOPPGGRF_1291', 'Point_NOPPGGRF_1309', 'Point_NOPPGGRF_1327', 'Point_NOPPGGRF_1399', 'Point_NOPPGGRF_1417', 
          'Point_NOPPGGRF_1435', 'Point_PPGGRF_1', 'Point_PPGGRF_3', 'Point_PPGGRF_5', 'Point_PPGGRF_7', 'Point_PPGGRF_10', 'Point_PPGGRF_11', 
          'Point_PPGGRF_14', 'Point_PPGGRF_41', 'Point_PPGGRF_45', 'Point_PPGGRF_68', 'Point_PPGGRF_76', 'Point_PPGGRF_78', 'Point_PPGGRF_86', 
          'Point_PPGGRF_120', 'Point_PPGGRF_124', 'Point_PPGGRF_130', 'Point_PPGGRF_132', 'Point_PPGGRF_134', 'Point_PPGGRF_136', 'Point_PPGGRF_138', 
          'Point_PPGGRF_142', 'Point_PPGGRF_148', 'Point_PPGGRF_191', 'Point_PPGGRF_193', 'Point_PPGGRF_235', 'Point_PPGGRF_237', 'Point_PPGGRF_239', 
          'Point_PPGGRF_245', 'Point_PPGGRF_247', 'Point_PPGGRF_251', 'Point_PPGGRF_253', 'Point_PPGGRF_257', 'Point_PPGGRF_259', 'Point_PPGGRF_261',
          'Point_PPGGRF_263', 'Point_PPGGRF_265', 'Point_PPGGRF_267', 'Point_PPGGRF_269', 'Point_PPGGRF_271', 'Point_PPGGRF_333', 'Point_PPGGRF_335', 
          'Point_PPGGRF_339', 'Point_PPGGRF_341', 'Point_PPGGRF_345', 'Point_PPGGRF_471', 'Point_PPGGRF_475', 'Point_PPGGRF_479', 'Point_PPGGRF_547',
          'Point_PPGGRF_549', 'Point_PPGGRF_613', 'Point_PPGGRF_619', 'Point_PPGGRF_621', 'Point_PPGGRF_690', 'Point_PPGGRF_695', 'Point_PPGGRF_765',
          'Point_PPGGRF_1291', 'Point_PPGGRF_1309', 'Point_PPGGRF_1327', 'Point_PPGGRF_1417', 'Point_PPGGRF_1435']

fields_line = ['Line_NOPPGGRF_68', 'Line_NOPPGGRF_78', 'Line_NOPPGGRF_549', 'Line_PPGGRF_68', 'Line_PPGGRF_78']

# Dictionary to store totals
totals = {"CT": 0, "poly": 0, "point": 0, "line": 0}

# Sum values for each category
for category, fields in zip(totals.keys(), [fields_CT, fields_poly, fields_point, fields_line]):
    with arcpy.da.SearchCursor(Funding_Dist, fields) as cursor:
        for row in cursor:
            for value in row:
                if value is not None:  # Ensure value is not NULL
                    totals[category] += value

# Print individual totals
grand_total = 0
for category, total in totals.items():
    total = round(total)
    grand_total += total
    print(f"The total summed calculated {category} field is: {total:,}")

# Print grand total
print(f"The grand total of all four fields is: {grand_total:,}")

The total summed calculated CT field is: 1,486,278,349
The total summed calculated poly field is: 126,206,277
The total summed calculated point field is: 10,904,132,106
The total summed calculated line field is: 258,591,471
The grand total of all four fields is: 12,775,208,203


### Save 'wide' version of the feature layer

In [118]:
# Path to save the 'wide' version of the feature class
Funding_Dist_wide = f"{project_gdb}\\Funding_Dist_{suffix}_wide"

# Step 1: Create a copy of the feature class
try:
    arcpy.CopyFeatures_management(Funding_Dist, Funding_Dist_wide)
    print(f"Feature class copied successfully to {Funding_Dist_wide}")
except Exception as e:
    print(f"An error occurred while copying the feature class: {e}")

Feature class copied successfully to C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Funding_Distribution_AR25.gdb\Funding_Dist_AR25_wide


In [119]:
#Rename GGRF field

arcpy.AlterField_management(
    in_table=Funding_Dist_wide,
    field="GGRF",
    new_field_name="Funding_Amount",
    new_field_alias="Funding Amount"
)

<Result 'C:\\Users\\gcowan\\Documents\\ArcGIS\\Projects\\Distribution Analysis\\Data Restructure\\Funding_Distribution_AR25\\Funding_Distribution_AR25.gdb\\Funding_Dist_AR25_wide'>

In [120]:
# delete unneeded fields
    # update point; line; CT fields from list above

# List of fields to delete for "wide"
fields_to_delete = ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 
                    'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 
                    'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 'CT_PPGGRF_128', 
                    'CT_PPGGRF_189', 'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837', 'CT_NOPPGGRF_TOTAL', 'CT_PPGGRF_TOTAL', 
                    'CT_GGRF_TOTAL', 'Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78', 'Poly_NOPPGGRF_86', 'Poly_NOPPGGRF_251', 
                    'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549', 'Poly_NOPPGGRF_695', 'Poly_PPGGRF_7', 
                    'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 'Poly_PPGGRF_265', 'Poly_PPGGRF_695', 
                    'Poly_NOPPGGRF_TOTAL', 'Poly_PPGGRF_TOTAL', 'Poly_GGRF_TOTAL', 'Point_NOPPGGRF_1', 'Point_NOPPGGRF_2',
                    'Point_NOPPGGRF_3', 'Point_NOPPGGRF_5', 'Point_NOPPGGRF_11', 'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41',
                    'Point_NOPPGGRF_45', 'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 'Point_NOPPGGRF_76', 'Point_NOPPGGRF_78',
                    'Point_NOPPGGRF_86', 'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132', 'Point_NOPPGGRF_134',
                    'Point_NOPPGGRF_136', 'Point_NOPPGGRF_138', 'Point_NOPPGGRF_142', 'Point_NOPPGGRF_148',
                    'Point_NOPPGGRF_191', 'Point_NOPPGGRF_193', 'Point_NOPPGGRF_235', 'Point_NOPPGGRF_237',
                    'Point_NOPPGGRF_239', 'Point_NOPPGGRF_243', 'Point_NOPPGGRF_245', 'Point_NOPPGGRF_247',
                    'Point_NOPPGGRF_249', 'Point_NOPPGGRF_251', 'Point_NOPPGGRF_253', 'Point_NOPPGGRF_255', 
                    'Point_NOPPGGRF_257', 'Point_NOPPGGRF_259', 'Point_NOPPGGRF_261', 'Point_NOPPGGRF_263', 
                    'Point_NOPPGGRF_265', 'Point_NOPPGGRF_331', 'Point_NOPPGGRF_333', 'Point_NOPPGGRF_335',
                    'Point_NOPPGGRF_339', 'Point_NOPPGGRF_341', 'Point_NOPPGGRF_345', 'Point_NOPPGGRF_471',
                    'Point_NOPPGGRF_475', 'Point_NOPPGGRF_477', 'Point_NOPPGGRF_479', 'Point_NOPPGGRF_547',
                    'Point_NOPPGGRF_549', 'Point_NOPPGGRF_613', 'Point_NOPPGGRF_615', 'Point_NOPPGGRF_617', 
                    'Point_NOPPGGRF_619', 'Point_NOPPGGRF_621', 'Point_NOPPGGRF_690', 'Point_NOPPGGRF_765', 
                    'Point_NOPPGGRF_839', 'Point_NOPPGGRF_981', 'Point_NOPPGGRF_1140', 'Point_NOPPGGRF_1144', 
                    'Point_NOPPGGRF_1291', 'Point_NOPPGGRF_1309', 'Point_NOPPGGRF_1327', 'Point_NOPPGGRF_1399', 
                    'Point_NOPPGGRF_1417', 'Point_NOPPGGRF_1435', 'Point_PPGGRF_1', 'Point_PPGGRF_3', 'Point_PPGGRF_5', 
                    'Point_PPGGRF_7', 'Point_PPGGRF_10', 'Point_PPGGRF_11', 'Point_PPGGRF_14', 'Point_PPGGRF_41', 
                    'Point_PPGGRF_45', 'Point_PPGGRF_68', 'Point_PPGGRF_76', 'Point_PPGGRF_78', 'Point_PPGGRF_86',
                    'Point_PPGGRF_120', 'Point_PPGGRF_124', 'Point_PPGGRF_130', 'Point_PPGGRF_132', 'Point_PPGGRF_134', 
                    'Point_PPGGRF_136', 'Point_PPGGRF_138', 'Point_PPGGRF_142', 'Point_PPGGRF_148', 'Point_PPGGRF_191', 
                    'Point_PPGGRF_193', 'Point_PPGGRF_235', 'Point_PPGGRF_237', 'Point_PPGGRF_239', 'Point_PPGGRF_245',
                    'Point_PPGGRF_247', 'Point_PPGGRF_251', 'Point_PPGGRF_253', 'Point_PPGGRF_257', 'Point_PPGGRF_259', 
                    'Point_PPGGRF_261', 'Point_PPGGRF_263', 'Point_PPGGRF_265', 'Point_PPGGRF_267', 'Point_PPGGRF_269', 
                    'Point_PPGGRF_271', 'Point_PPGGRF_333', 'Point_PPGGRF_335', 'Point_PPGGRF_339', 'Point_PPGGRF_341', 
                    'Point_PPGGRF_345', 'Point_PPGGRF_471', 'Point_PPGGRF_475', 'Point_PPGGRF_479', 'Point_PPGGRF_547', 
                    'Point_PPGGRF_549', 'Point_PPGGRF_613', 'Point_PPGGRF_619', 'Point_PPGGRF_621', 'Point_PPGGRF_690',
                    'Point_PPGGRF_695', 'Point_PPGGRF_765', 'Point_PPGGRF_1291', 'Point_PPGGRF_1309', 'Point_PPGGRF_1327',
                    'Point_PPGGRF_1417', 'Point_PPGGRF_1435', 'Point_NOPPGGRF_Total', 'Point_PPGGRF_Total', 
                    'Point_GGRF_Total', 'Point_Count', 'Line_NOPPGGRF_68', 'Line_NOPPGGRF_78', 'Line_NOPPGGRF_549', 
                    'Line_PPGGRF_68', 'Line_PPGGRF_78', 'Line_NOPPGGRF_Total', 'Line_PPGGRF_Total', 'Line_GGRF_Total']

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist_wide, fields_to_delete)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


### Save a Census Tract groupby CSV of Wide Data Set

In [121]:
# Input feature class
fc = Funding_Dist

# Convert to pandas DataFrame
fields = [f.name for f in arcpy.ListFields(fc) if f.type not in ("Geometry", "OID")]
data = [row for row in arcpy.da.SearchCursor(fc, fields)]
df_wide = pd.DataFrame(data, columns=fields)

# Grouping field
group_field = ["Census_Tract"]

# Fields to sum
fields_to_sum = [  # (keep this list as-is)
    "GGRF", 'sum_group_1', 'sum_group_2', 'sum_group_3', 'sum_group_4', 'sum_group_5', 'sum_group_7', 'sum_group_8',
    'sum_group_9', 'sum_group_10', 'sum_group_11', 'sum_group_14', 'sum_group_41', 'sum_group_45', 'sum_group_68',
    'sum_group_72', 'sum_group_76', 'sum_group_78', 'sum_group_80', 'sum_group_82', 'sum_group_86', 'sum_group_88',
    'sum_group_90', 'sum_group_120', 'sum_group_124', 'sum_group_128', 'sum_group_130', 'sum_group_132', 'sum_group_134',
    'sum_group_136', 'sum_group_138', 'sum_group_142', 'sum_group_148', 'sum_group_189', 'sum_group_191', 'sum_group_193',
    'sum_group_235', 'sum_group_237', 'sum_group_239', 'sum_group_243', 'sum_group_245', 'sum_group_247', 'sum_group_249',
    'sum_group_251', 'sum_group_253', 'sum_group_255', 'sum_group_257', 'sum_group_259', 'sum_group_261', 'sum_group_263',
    'sum_group_265', 'sum_group_267', 'sum_group_331', 'sum_group_333', 'sum_group_335', 'sum_group_337', 'sum_group_339',
    'sum_group_341', 'sum_group_345', 'sum_group_471', 'sum_group_475', 'sum_group_477', 'sum_group_479', 'sum_group_549',
    'sum_group_613', 'sum_group_615', 'sum_group_617', 'sum_group_619', 'sum_group_621', 'sum_group_690', 'sum_group_695',
    'sum_group_765', 'sum_group_837', 'sum_group_839', 'sum_group_981', 'sum_group_1136', 'sum_group_1140', 'sum_group_1144',
    'sum_group_1291', 'sum_group_1309', 'sum_group_1327', 'sum_group_1399', 'sum_group_1417', 'sum_group_1435'
]

# Fields to copy (deduplicated)
fields_to_copy = list(set([
    "PctBIPOC", "BIPOCMaj", "DemogMaj", "CES4range", "CES4decile", 'CCIquartile',
    "County", "ApproxLoc", "Child_10", "Pop_10_64", "Elderly_65", "Hispanic", "White", "African_American",
    "Native_American", "Other_Multiple", "Pacific_Islander", "Asian_American", "CIscoreP", "Ozone_Pctl",
    "PM2_5_Pctl", "Diesel_PM_Pctl", "Pesticides_Pctl", "Tox_Releases_Pctl", "Traffic_Pctl", "DrinkingWaterPctl",
    "Lead_Pctl", "Cleanups_Pctl", "GW_Threats_Pctl", "Haz_Waste_Pctl", "ImpWaterBodPctl", "Solid_Waste_Pctl",
    "Pollution_Pctl", "Asthma_Pctl", "LowBirthW_Pctl", "Cardiovasc_Pctl", "Education_Pctl", "Ling_Isol_Pctl",
    "Poverty_Pctl", "Unemploy_Pctl", "HousBurd_Pctl", "PopCharP", "Tribal_Name"
]))

# Define aggregation rules
agg_dict = {field: 'sum' for field in fields_to_sum}
for field in fields_to_copy:
    if field == "Name":
        agg_dict[field] = lambda x: next((val for val in x if pd.notnull(val) and str(val).strip() != ""), None)
    else:
        agg_dict[field] = 'first'

# Group and aggregate
funding_CT_wide = df_wide.groupby(group_field, as_index=False).agg(agg_dict)

# Preview sample
funding_CT_wide.sample(10)

,Census_Tract,GGRF,sum_group_1,sum_group_2,sum_group_3,sum_group_4,sum_group_5,sum_group_7,sum_group_8,sum_group_9,...,HousBurd_Pctl,PctBIPOC,Asthma_Pctl,ImpWaterBodPctl,Cleanups_Pctl,Poverty_Pctl,BIPOCMaj,DrinkingWaterPctl,Diesel_PM_Pctl,County
7820,6.111000e+09,552510.833231,0.0,0.0,0.0,10000.0,0.0,0.0,0.0,0.0,...,54.626109,11.6129,8.624128,94.561109,81.914894,28.165829,BIPOC,76.445610,0.062228,Ventura
2864,6.037543e+09,92200.827499,0.0,0.0,0.0,15000.0,0.0,0.0,7500.0,0.0,...,79.911280,97.4161,93.058325,66.736665,17.077268,73.630653,BIPOC,50.405895,31.362788,Los Angeles
1185,6.037104e+09,317476.596026,0.0,0.0,0.0,68500.0,0.0,0.0,19000.0,5000.0,...,76.869455,94.3013,83.536889,0.000000,31.243001,73.316583,BIPOC,83.127264,50.043559,Los Angeles
6271,6.073021e+09,305335.395250,0.0,0.0,0.0,63000.0,0.0,0.0,0.0,0.0,...,1.406844,31.3450,6.069292,23.876522,0.000000,9.271357,BIPOC,23.466966,7.741133,San Diego
1736,6.037209e+09,85872.468588,0.0,0.0,0.0,71500.0,0.0,0.0,0.0,5000.0,...,85.500634,93.0399,78.501994,0.000000,62.672639,95.414573,BIPOC,92.531535,99.639079,Los Angeles
6307,6.075013e+09,65552.315479,0.0,0.0,0.0,63000.0,0.0,0.0,0.0,0.0,...,38.783270,24.1210,4.012961,86.959261,93.542367,8.530151,BIPOC,15.036843,96.938395,San Francisco
2997,6.037570e+09,581296.778932,0.0,0.0,0.0,21000.0,0.0,0.0,9000.0,0.0,...,93.726236,96.5231,87.387836,82.969341,25.625233,89.761307,BIPOC,31.172724,97.548227,Los Angeles
2710,6.037531e+09,55187.579126,0.0,0.0,0.0,25500.0,0.0,0.0,14000.0,0.0,...,62.420786,98.8021,65.191924,0.000000,76.801045,84.698492,BIPOC,75.134258,55.532047,Los Angeles
2092,6.037276e+09,162156.202428,0.0,0.0,0.0,132312.0,24610.0,0.0,0.0,0.0,...,80.468948,57.1541,38.758724,72.154557,60.377006,21.721106,BIPOC,43.387036,81.754823,Los Angeles
3194,6.037620e+09,234246.762841,0.0,0.0,0.0,176000.0,23000.0,0.0,19000.0,0.0,...,27.782003,37.9616,12.761715,51.217976,95.464726,22.474874,BIPOC,8.954665,83.273180,Los Angeles


In [122]:
funding_CT_wide = funding_CT_wide.rename(columns={'GGRF': 'Funding_Amount'})

In [123]:
# check sum of GGRF total after groupby - QC

total = funding_CT_wide["Funding_Amount"].sum()
print(f"{total:,}")

12,775,208,202.749279


In [124]:
# check sum of GGRF total before groupby (to compare to above - QC)
total = df_wide["GGRF"].sum()
print(f"{total:,}")

12,775,208,202.749279


In [125]:
# Export to a CSV

# Define the output CSV file path
output_csv = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_Wide_CT_Tabular.csv"

# Save the DataFrame to CSV
funding_CT_wide.to_csv(output_csv, index=False)

print(f"Data saved to: {output_csv}")

Data saved to: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_Wide_CT_Tabular.csv


## Create 'Long' File Version

### Restructure the file

In [126]:
# add a wide record unique ID to the layer

# Add a new field for the unique ID
unique_id_field = "UniqueID_wide"
arcpy.management.AddField(Funding_Dist, unique_id_field, "LONG")

# Calculate the unique ID based on OBJECTID
arcpy.management.CalculateField(Funding_Dist, unique_id_field, "!OBJECTID!", "PYTHON3")

print(f"Unique ID field '{unique_id_field}' added and populated successfully.")

Unique ID field 'UniqueID_wide' added and populated successfully.


In [127]:
# restructure the data to the long format

# Path to save the 'long' version of the feature class
Funding_Dist_long = f"{project_gdb}\\Funding_Dist_{suffix}_long"

# List of funding columns
funding_columns = ['sum_group_1', 'sum_group_2', 'sum_group_3', 'sum_group_4', 'sum_group_5', 'sum_group_7', 'sum_group_8',
    'sum_group_9', 'sum_group_10', 'sum_group_11', 'sum_group_14', 'sum_group_41', 'sum_group_45', 'sum_group_68',
    'sum_group_72', 'sum_group_76', 'sum_group_78', 'sum_group_80', 'sum_group_82', 'sum_group_86', 'sum_group_88',
    'sum_group_90', 'sum_group_120', 'sum_group_124', 'sum_group_128', 'sum_group_130', 'sum_group_132', 'sum_group_134',
    'sum_group_136', 'sum_group_138', 'sum_group_142', 'sum_group_148', 'sum_group_189', 'sum_group_191', 'sum_group_193',
    'sum_group_235', 'sum_group_237', 'sum_group_239', 'sum_group_243', 'sum_group_245', 'sum_group_247', 'sum_group_249',
    'sum_group_251', 'sum_group_253', 'sum_group_255', 'sum_group_257', 'sum_group_259', 'sum_group_261', 'sum_group_263',
    'sum_group_265', 'sum_group_267', 'sum_group_331', 'sum_group_333', 'sum_group_335', 'sum_group_337', 'sum_group_339',
    'sum_group_341', 'sum_group_345', 'sum_group_471', 'sum_group_475', 'sum_group_477', 'sum_group_479', 'sum_group_549',
    'sum_group_613', 'sum_group_615', 'sum_group_617', 'sum_group_619', 'sum_group_621', 'sum_group_690', 'sum_group_695',
    'sum_group_765', 'sum_group_837', 'sum_group_839', 'sum_group_981', 'sum_group_1136', 'sum_group_1140', 'sum_group_1144',
    'sum_group_1291', 'sum_group_1309', 'sum_group_1327', 'sum_group_1399', 'sum_group_1417', 'sum_group_1435']

# Get all the fields in the input shapefile
all_fields = [f.name for f in arcpy.ListFields(Funding_Dist) if f.type not in ("Geometry", "OID")]

# Create a copy of the input feature class
arcpy.management.CopyFeatures(Funding_Dist, Funding_Dist_long)

# Add new fields to the output shapefile
arcpy.AddField_management(Funding_Dist_long, 'Program_Name', 'TEXT', field_length=100)
arcpy.AddField_management(Funding_Dist_long, 'Funding_Amount', 'DOUBLE')

# Open an InsertCursor for the output feature class
with arcpy.da.InsertCursor(Funding_Dist_long, all_fields + ['Program_Name', 'Funding_Amount', 'SHAPE@']) as insert_cursor:
    # Open a SearchCursor to iterate through the input shapefile
    with arcpy.da.SearchCursor(Funding_Dist, all_fields + funding_columns + ['SHAPE@']) as search_cursor:
        for row in search_cursor:
            # Extract values for fields that should remain unchanged
            original_values = list(row[:len(all_fields)])
            geometry = row[-1]  # Extract the geometry
            
            # Iterate through each funding column
            for idx, funding_column in enumerate(funding_columns, start=len(all_fields)):
                funding_amount = row[idx]  # Access the funding amount from the corresponding column
                
                # Insert a new row if the funding amount is greater than 0
                if funding_amount > 0:
                    insert_cursor.insertRow(original_values + [funding_column, funding_amount, geometry])

print("Restructuring completed.")

Restructuring completed.


### Check program totals

In [128]:
# check program total sum of individual summed fields for long version 
    # paste this list from above output

# Define the columns to sum
funding_columns = [
'sum_group_1', 'sum_group_2', 'sum_group_3', 'sum_group_4', 'sum_group_5', 'sum_group_7', 'sum_group_8',
    'sum_group_9', 'sum_group_10', 'sum_group_11', 'sum_group_14', 'sum_group_41', 'sum_group_45', 'sum_group_68',
    'sum_group_72', 'sum_group_76', 'sum_group_78', 'sum_group_80', 'sum_group_82', 'sum_group_86', 'sum_group_88',
    'sum_group_90', 'sum_group_120', 'sum_group_124', 'sum_group_128', 'sum_group_130', 'sum_group_132', 'sum_group_134',
    'sum_group_136', 'sum_group_138', 'sum_group_142', 'sum_group_148', 'sum_group_189', 'sum_group_191', 'sum_group_193',
    'sum_group_235', 'sum_group_237', 'sum_group_239', 'sum_group_243', 'sum_group_245', 'sum_group_247', 'sum_group_249',
    'sum_group_251', 'sum_group_253', 'sum_group_255', 'sum_group_257', 'sum_group_259', 'sum_group_261', 'sum_group_263',
    'sum_group_265', 'sum_group_267', 'sum_group_331', 'sum_group_333', 'sum_group_335', 'sum_group_337', 'sum_group_339',
    'sum_group_341', 'sum_group_345', 'sum_group_471', 'sum_group_475', 'sum_group_477', 'sum_group_479', 'sum_group_549',
    'sum_group_613', 'sum_group_615', 'sum_group_617', 'sum_group_619', 'sum_group_621', 'sum_group_690', 'sum_group_695',
    'sum_group_765', 'sum_group_837', 'sum_group_839', 'sum_group_981', 'sum_group_1136', 'sum_group_1140', 'sum_group_1144',
    'sum_group_1291', 'sum_group_1309', 'sum_group_1327', 'sum_group_1399', 'sum_group_1417', 'sum_group_1435'
]

# Initialize total sum
total_sum = 0

# Use a search cursor to iterate over the records and sum the values
with arcpy.da.SearchCursor(Funding_Dist, funding_columns) as cursor:
    for row in cursor:
        for value in row:
            if value is not None:  # Ensure value is not NULL
                total_sum += value

# Print the final summed total
print(f"The total sum calculated for all funding fields is: {total_sum:,}")

The total sum calculated for all funding fields is: 12,775,208,202.74934


In [129]:
# check sum of final program total column

# Define the column to sum
funding_column = "Funding_Amount"

# Initialize total sum
total_sum = 0

# Use a search cursor to iterate over the records and sum the values
with arcpy.da.SearchCursor(Funding_Dist_long, [funding_column]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Ensure value is not NULL
            total_sum += row[0]

# Print the final summed total
print(f"The total sum calculated for the program funding field is: {total_sum:,}")

The total sum calculated for the program funding field is: 12,775,208,202.74934


### Add "all CA" rows

In [130]:
# Use the original columns from the "wide" format to fill in the new ALL CA cateogry in the Funding_Amount and Project_Name fields
target_field = "Funding_Amount"
source_field = "GGRF"
constant_field = "Program_Name"
constant_value = "All California"

with arcpy.da.UpdateCursor(Funding_Dist_long, [target_field, source_field, constant_field]) as cursor:
    for row in cursor:
        if row[0] in [None, "", " "]:
            row[0] = row[1]
            row[2] = constant_value  # set FieldC
            cursor.updateRow(row)

print(f"{target_field} updated from {source_field} where blank, and {constant_field} set to '{constant_value}'.")


Funding_Amount updated from GGRF where blank, and Program_Name set to 'All California'.


### Delete unneeded columns & rows

In [131]:
# Delete the individual funding columns
for column in funding_columns:
    if arcpy.ListFields(Funding_Dist_long, column): 
        arcpy.DeleteField_management(Funding_Dist_long, column)

# Print completion message
print("All specified funding columns have been processed and deleted.")

All specified funding columns have been processed and deleted.


In [132]:
# delete additional unneeded fields

# List of fields to delete for "long" version (UPDATE with information from above re: new fields)
fields_to_delete = ['CT_NOPPGGRF_4', 'CT_NOPPGGRF_8', 'CT_NOPPGGRF_9', 'CT_NOPPGGRF_80', 'CT_NOPPGGRF_82', 'CT_NOPPGGRF_88', 
                    'CT_NOPPGGRF_90', 'CT_NOPPGGRF_128', 'CT_NOPPGGRF_337', 'CT_NOPPGGRF_1136', 'CT_PPGGRF_4', 'CT_PPGGRF_8', 
                    'CT_PPGGRF_9', 'CT_PPGGRF_80', 'CT_PPGGRF_82', 'CT_PPGGRF_88', 'CT_PPGGRF_90', 'CT_PPGGRF_128', 
                    'CT_PPGGRF_189', 'CT_PPGGRF_329', 'CT_PPGGRF_337', 'CT_PPGGRF_837', 'Poly_NOPPGGRF_68', 'Poly_NOPPGGRF_78',
                    'Poly_NOPPGGRF_86', 'Poly_NOPPGGRF_251', 'Poly_NOPPGGRF_265', 'Poly_NOPPGGRF_345', 'Poly_NOPPGGRF_549',
                    'Poly_NOPPGGRF_695', 'Poly_PPGGRF_7', 'Poly_PPGGRF_68', 'Poly_PPGGRF_78', 'Poly_PPGGRF_251', 
                    'Poly_PPGGRF_265', 'Poly_PPGGRF_695', 'Point_NOPPGGRF_1', 'Point_NOPPGGRF_2', 'Point_NOPPGGRF_3', 
                    'Point_NOPPGGRF_5', 'Point_NOPPGGRF_11', 'Point_NOPPGGRF_14', 'Point_NOPPGGRF_41', 'Point_NOPPGGRF_45', 
                    'Point_NOPPGGRF_68', 'Point_NOPPGGRF_72', 'Point_NOPPGGRF_76', 'Point_NOPPGGRF_78', 'Point_NOPPGGRF_86',
                    'Point_NOPPGGRF_124', 'Point_NOPPGGRF_132', 'Point_NOPPGGRF_134', 'Point_NOPPGGRF_136', 'Point_NOPPGGRF_138',
                    'Point_NOPPGGRF_142', 'Point_NOPPGGRF_148', 'Point_NOPPGGRF_191', 'Point_NOPPGGRF_193', 'Point_NOPPGGRF_235',
                    'Point_NOPPGGRF_237', 'Point_NOPPGGRF_239', 'Point_NOPPGGRF_243', 'Point_NOPPGGRF_245', 'Point_NOPPGGRF_247',
                    'Point_NOPPGGRF_249', 'Point_NOPPGGRF_251', 'Point_NOPPGGRF_253', 'Point_NOPPGGRF_255', 'Point_NOPPGGRF_257',
                    'Point_NOPPGGRF_259', 'Point_NOPPGGRF_261', 'Point_NOPPGGRF_263', 'Point_NOPPGGRF_265', 'Point_NOPPGGRF_331',
                    'Point_NOPPGGRF_333', 'Point_NOPPGGRF_335', 'Point_NOPPGGRF_339', 'Point_NOPPGGRF_341', 'Point_NOPPGGRF_345',
                    'Point_NOPPGGRF_471', 'Point_NOPPGGRF_475', 'Point_NOPPGGRF_477', 'Point_NOPPGGRF_479', 'Point_NOPPGGRF_547',
                    'Point_NOPPGGRF_549', 'Point_NOPPGGRF_613', 'Point_NOPPGGRF_615', 'Point_NOPPGGRF_617', 'Point_NOPPGGRF_619',
                    'Point_NOPPGGRF_621', 'Point_NOPPGGRF_690', 'Point_NOPPGGRF_765', 'Point_NOPPGGRF_839', 'Point_NOPPGGRF_981',
                    'Point_NOPPGGRF_1140', 'Point_NOPPGGRF_1144', 'Point_NOPPGGRF_1291', 'Point_NOPPGGRF_1309',
                    'Point_NOPPGGRF_1327', 'Point_NOPPGGRF_1399', 'Point_NOPPGGRF_1417', 'Point_NOPPGGRF_1435', 
                    'Point_PPGGRF_1', 'Point_PPGGRF_3', 'Point_PPGGRF_5', 'Point_PPGGRF_7', 'Point_PPGGRF_10', 
                    'Point_PPGGRF_11', 'Point_PPGGRF_14', 'Point_PPGGRF_41', 'Point_PPGGRF_45', 'Point_PPGGRF_68', 
                    'Point_PPGGRF_76', 'Point_PPGGRF_78', 'Point_PPGGRF_86', 'Point_PPGGRF_120', 'Point_PPGGRF_124',
                    'Point_PPGGRF_130', 'Point_PPGGRF_132', 'Point_PPGGRF_134', 'Point_PPGGRF_136', 'Point_PPGGRF_138',
                    'Point_PPGGRF_142', 'Point_PPGGRF_148', 'Point_PPGGRF_191', 'Point_PPGGRF_193', 'Point_PPGGRF_235', 
                    'Point_PPGGRF_237', 'Point_PPGGRF_239', 'Point_PPGGRF_245', 'Point_PPGGRF_247', 'Point_PPGGRF_251',
                    'Point_PPGGRF_253', 'Point_PPGGRF_257', 'Point_PPGGRF_259', 'Point_PPGGRF_261', 'Point_PPGGRF_263', 
                    'Point_PPGGRF_265', 'Point_PPGGRF_267', 'Point_PPGGRF_269', 'Point_PPGGRF_271', 'Point_PPGGRF_333',
                    'Point_PPGGRF_335', 'Point_PPGGRF_339', 'Point_PPGGRF_341', 'Point_PPGGRF_345', 'Point_PPGGRF_471', 
                    'Point_PPGGRF_475', 'Point_PPGGRF_479', 'Point_PPGGRF_547', 'Point_PPGGRF_549', 'Point_PPGGRF_613',
                    'Point_PPGGRF_619', 'Point_PPGGRF_621', 'Point_PPGGRF_690', 'Point_PPGGRF_695', 'Point_PPGGRF_765',
                    'Point_PPGGRF_1291', 'Point_PPGGRF_1309', 'Point_PPGGRF_1327', 'Point_PPGGRF_1417', 'Point_PPGGRF_1435',
                    'Line_NOPPGGRF_68', 'Line_NOPPGGRF_78', 'Line_NOPPGGRF_549', 'Line_PPGGRF_68', 'Line_PPGGRF_78', 'CCIpc',
                    'CCIpcPct', 'CCIpcDecile', 'CCIpcQuartile','GGRF']

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist_long, fields_to_delete)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


### Add additional fields needed for 'long' version

#### Demographic percentage range columns

In [133]:
# Add demographic percentage columns
demographic_columns = ["Hispanic", "White", "African_American", "Native_American", "Other_Multiple", 
                       "Pacific_Islander", "Asian_American"]

# Add new fields to hold the range label for each demographic column
for column in demographic_columns:
    range_field_name = f"{column}_Range"
    arcpy.AddField_management(Funding_Dist_long, range_field_name, "TEXT", field_length=20)

# Populate the new fields with the appropriate range label
with arcpy.da.UpdateCursor(Funding_Dist_long, demographic_columns + [f"{col}_Range" for col in demographic_columns]) as cursor:
    for row in cursor:
        for col_index, column in enumerate(demographic_columns):
            percent_value = row[col_index]
            if percent_value is not None:
                # Determine the range the value falls into
                for i in range(1, 11):
                    range_start = (i - 1) * 10
                    range_end = i * 10
                    if range_start <= percent_value < range_end or (i == 10 and percent_value == 100): 
                        range_label = f"{range_start}-{range_end}%"
                        range_field_index = len(demographic_columns) + col_index
                        row[range_field_index] = range_label
                        break
            else:
                # If the value is None, leave the range field blank
                range_field_index = len(demographic_columns) + col_index
                row[range_field_index] = None
        cursor.updateRow(row)

print("Demographic range fields updated successfully.")

Demographic range fields updated successfully.


#### Per capita column (updated from 'wide' version)

In [134]:
# Update the per capita column for "long" version

input_field = "Funding_Amount"
population_field = "Population"
per_capita_field = "CCIpc_prgm"

# Add the new per capita field 
if per_capita_field not in [f.name for f in arcpy.ListFields(Funding_Dist_long)]:
    arcpy.AddField_management(Funding_Dist_long, per_capita_field, "DOUBLE") 

# Calculate per capita funding
with arcpy.da.UpdateCursor(Funding_Dist_long, [input_field, population_field, per_capita_field]) as cursor:
    for row in cursor:
        funding = row[0]
        population = row[1]

        # Check if population is valid and non-zero
        if population and population > 0:
            row[2] = funding / population  # Calculate per capita funding
        else:
            row[2] = None  # Assign NULL if population is 0 or None

        cursor.updateRow(row)

print(f"Per capita funding values have been calculated and stored in the field '{per_capita_field}'.")

Per capita funding values have been calculated and stored in the field 'CCIpc_prgm'.


#### Program names

In [135]:
# Map old names in Program_Name to new descriptive names
    # *Update with new program names* 
column_mapping = {
"All California": "<All Programs>", 
"sum_group_1": "Dairy Digester Research and Development Program",
"sum_group_2": "Renewable and Alternative Fuels",
"sum_group_3": "State Water Efficiency and Enhancement Program",
"sum_group_4": "Clean Vehicle Rebate Project",
"sum_group_5": "Clean Truck and Bus Vouchers (HVIP)",
"sum_group_7": "Clean Mobility Options",
"sum_group_8": "Clean Cars 4 All",
"sum_group_9": "Financing Assistance for Lower-Income Consumers",
"sum_group_10":  "Advanced Technology Demonstration and Pilot Projects",
"sum_group_11": "Zero-Emission Truck and Bus Pilot",
"sum_group_14": "Sustainable Agricultural Lands Conservation Program",
"sum_group_41": "Affordable Housing and Sustainable Communities Program",
"sum_group_45": "Urban and Community Forestry",
"sum_group_68": "Low Carbon Transit Operations Program",
"sum_group_72": "State Water Project Turbines",
"sum_group_76": "Wetlands and Watershed Restoration",
"sum_group_78": "Transit and Intercity Rail Capital Program",
"sum_group_80": "Single-Family Solar Photovoltaics (PV)",
"sum_group_82": "Water-Energy Grant Program",
"sum_group_86": "Forest Health Program",
"sum_group_88": "Single-Family Energy Efficiency and Solar PV",
"sum_group_90": "Multi-Family Energy Efficiency and Renewables",
"sum_group_120": "Agricultural Worker Vanpools",
"sum_group_124": "Rural School Bus Pilot Projects",
"sum_group_128": "Woodsmoke Reduction Program",
"sum_group_130": "Active Transportation Program",
"sum_group_132": "Organics and Recycling Loans",
"sum_group_134": "Recycled Fiber, Plastic, and Glass Grant Program",
"sum_group_136": "Food Waste Prevention and Rescue Grants",
"sum_group_138": "Urban Greening Program",
"sum_group_142":  "Transformative Climate Communities",
"sum_group_148": "Organics Grants",
"sum_group_189": "Community Solar Pilot",
"sum_group_191": "Alternative Manure Management Program",
"sum_group_193": "Healthy Soils Program",
"sum_group_235": "Community Air Protection Incentives",
"sum_group_237": "Funding Agricultural Replacement Measures for Emission Reductions Program",
"sum_group_239": "Coastal Resilience Planning",
"sum_group_243": "Climate Change Adaptation and Coastal Resilience Planning",
"sum_group_245": "Wildfire Prevention Program",
"sum_group_247": "Wildfire Prevention Grants Program",
"sum_group_249": "Wildfire Response and Readiness",
"sum_group_251": "Climate Adaptation and Resiliency Program",
"sum_group_253": "Climate Ready Program",
"sum_group_255": "Climate Change Research Program",
"sum_group_257": "Food Production Investment Program",
"sum_group_259": "Renewable Energy for Agriculture Program",
"sum_group_261": "Training and Workforce Development Program",
"sum_group_263": "Clean Off Road Equipment Voucher Incentive Project",
"sum_group_265": "Community Air Grants",
"sum_group_267": "Zero-and Near Zero-Emission Freight Facilities Project",
"sum_group_331": "Prescribed Fire and Smoke Monitoring Program",
"sum_group_333": "Fuels Reduction Crews", 
"sum_group_335": "Low-Carbon Fuel Production Program",
"sum_group_337": "​Farmworker Housing Energy Efficiency & Solar PV​",
"sum_group_339": "Regional Forest and Fire Capacity",
"sum_group_341": "Community Assistance for Climate Equity Program",
"sum_group_345": "Clean Mobility in Schools Project",
"sum_group_471": "Outreach, Education, and Awareness",
"sum_group_475": "Fluorinated Gases Emission Reduction Incentives",
"sum_group_477": "Transition to a Carbon-Neutral Economy",
"sum_group_479": "Forest Health Research and Monitoring",
"sum_group_549": "Safe and Affordable Drinking Water Fund",
"sum_group_613":  "High Road Training Partnerships",
"sum_group_615": "Fire Engines and Maintenance",
"sum_group_617": "AB617 Implementation Funds",
"sum_group_619": "Community Composting for Green Spaces Grant Program",
"sum_group_621": "Community Fire Planning and Preparedness",
"sum_group_690": "Reuse Grant Program",
"sum_group_695": "Sustainable Transportation Equity Project",
"sum_group_765": "Climate Smart Agriculture Technical Assistance Program",
"sum_group_837": "IDEAL ZEV Workforce Pilot Project",
"sum_group_839": "SB 1383 Local Assistance Grant Program",
"sum_group_981": "Co-Digestion Grant Program",
"sum_group_1136": "Statewide Mobile Monitoring Initiative",
"sum_group_1140": "California Schools Healthy Air, Plumbing and Efficiency Program (CALSHAPE)",
"sum_group_1144": "Sea Level Rise Adaptation Plans",
"sum_group_1291": "Demonstration State Forests",
"sum_group_1309": "Miscellaneous Support and Special Projects for CAL FIRE Vegetation Management and Incident Response",
"sum_group_1327": "Forest Legacy",
"sum_group_1399": "Business and Workforce Development",
"sum_group_1417": "California Forest Improvement Program",
"sum_group_1435": "Wildfire Resilience (Various)"}

# Update the Program_Name field
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Program_Name"]) as cursor:
    for row in cursor:
        if row[0] in column_mapping:
            row[0] = column_mapping[row[0]]  # Replace with the new name
            cursor.updateRow(row)

print("Program_Name values renamed successfully.")

Program_Name values renamed successfully.


In [136]:
# Final check of long version GGRF total

total = 0

with arcpy.da.SearchCursor(Funding_Dist_long, ["Funding_Amount"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            total += row[0]

print("Sum divided by 2: {:,}".format(total / 2))

Sum divided by 2: 12,775,208,202.7491


## Save both formats as layer packages

In [137]:
# Define paths
Funding_Dist_wide = f"{project_gdb}\\Funding_Dist_{suffix}_wide"
Funding_Dist_long = f"{project_gdb}\\Funding_Dist_{suffix}_long"

layer_long_name = "FeatureLayer_Long"
layer_wide_name = "FeatureLayer_Wide"

FundingDist_lpkx_long = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_long.lpkx"
FundingDist_lpkx_wide = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_wide.lpkx"

# Save 'long' version layer package
try:
    # Step 1: Create a feature layer
    arcpy.management.MakeFeatureLayer(Funding_Dist_long, layer_long_name)
    print(f"Feature layer '{layer_long_name}' created successfully.")

    # Step 2: Create a layer package
    arcpy.management.PackageLayer(layer_long_name, FundingDist_lpkx_long, "PRESERVE", "CONVERT_ARCSDE", "DEFAULT", "ALL")
    print(f"Layer package saved successfully at: {FundingDist_lpkx_long}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Cleanup: Delete the temporary layer
    if arcpy.Exists(layer_long_name):
        arcpy.management.Delete(layer_long_name)
        print(f"Temporary layer '{layer_long_name}' deleted.")

# Save 'wide' version layer package
try:
    # Step 1: Create a feature layer
    arcpy.management.MakeFeatureLayer(Funding_Dist_wide, layer_wide_name)
    print(f"Feature layer '{layer_wide_name}' created successfully.")

    # Step 2: Create a layer package
    arcpy.management.PackageLayer(layer_wide_name, FundingDist_lpkx_wide, "PRESERVE", "CONVERT_ARCSDE", "DEFAULT", "ALL")
    print(f"Layer package saved successfully at: {FundingDist_lpkx_wide}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Cleanup: Delete the temporary layer
    if arcpy.Exists(layer_wide_name):
        arcpy.management.Delete(layer_wide_name)
        print(f"Temporary layer '{layer_wide_name}' deleted.")

Feature layer 'FeatureLayer_Long' created successfully.
Layer package saved successfully at: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_long.lpkx
Temporary layer 'FeatureLayer_Long' deleted.
Feature layer 'FeatureLayer_Wide' created successfully.
Layer package saved successfully at: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_wide.lpkx
Temporary layer 'FeatureLayer_Wide' deleted.


## Restructure Data to Aggregate by Census Tract

### Add additional fields needed for CT version

#### LIC Funding Field

In [154]:
# Name of the new field
new_field = "LIC_Funding"

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(Funding_Dist_long)]
if new_field not in field_names:
    arcpy.management.AddField(Funding_Dist_long, new_field, "DOUBLE")

# Keywords to search for (case-insensitive)
lic_keywords = ["low-income community", "Low-income community"]

# Update the new field where Designation includes any LIC keyword
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Designation", "Funding_Amount", new_field]) as cursor:
    for row in cursor:
        designation = (row[0] or "").lower()
        if any(keyword in designation for keyword in lic_keywords):
            row[2] = row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("LIC funding field added")

LIC funding field added


#### DAC Funding Field

In [155]:
new_field = "DAC_Funding"

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(Funding_Dist_long)]
if new_field not in field_names:
    arcpy.management.AddField(Funding_Dist_long, new_field, "DOUBLE")

# Update the new field where Designation includes "Low-Income Community"
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Designation", "Funding_Amount", new_field]) as cursor:
    for row in cursor:
        if "Disadvantaged" in (row[0] or ""):
            row[2] = row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("DAC funding field added")

DAC funding field added


#### DAC 1/2 Mile Neighbor LIC Funding Field

In [156]:
new_field = "DACN_LIC_Funding"

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(Funding_Dist_long)]
if new_field not in field_names:
    arcpy.management.AddField(Funding_Dist_long, new_field, "DOUBLE")

# Update the new field where Designation includes "Low-Income Community"
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Designation", "Funding_Amount", new_field]) as cursor:
    for row in cursor:
        if "DAC 1/2 mile neighbor: low-income community" in (row[0] or ""):
            row[2] = row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("DAC 1/2 mile LIC funding field added")

DAC 1/2 mile LIC funding field added


#### DAC 1/2 Neighbor LIH Funding Field

In [157]:
new_field = "DACN_LIH_Funding"

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(Funding_Dist_long)]
if new_field not in field_names:
    arcpy.management.AddField(Funding_Dist_long, new_field, "DOUBLE")

# Update the new field where Designation includes "Low-Income Community"
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Designation", "Funding_Amount", new_field]) as cursor:
    for row in cursor:
        if "DAC 1/2 mile neighbor: low-income household eligible" in (row[0] or ""):
            row[2] = row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("DAC 1/2 mile LIH funding field added")

DAC 1/2 mile LIH funding field added


#### Not a Priority Population Funding Field

In [158]:
new_field = "NOPP_Funding"

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(Funding_Dist_long)]
if new_field not in field_names:
    arcpy.management.AddField(Funding_Dist_long, new_field, "DOUBLE")

# Update the new field where Designation includes "Low-Income Community"
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Designation", "Funding_Amount", new_field]) as cursor:
    for row in cursor:
        if "Low-income household eligible" in (row[0] or ""):
            row[2] = row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("No PP funding field added")

No PP funding field added


#### Tribal Funding

In [159]:
new_field = "Tribal_Funding"

# Add the new field if it doesn't exist
field_names = [f.name for f in arcpy.ListFields(Funding_Dist_long)]
if new_field not in field_names:
    arcpy.management.AddField(Funding_Dist_long, new_field, "DOUBLE")

# Update the new field where Designation includes "Low-Income Community"
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Designation", "Funding_Amount", new_field]) as cursor:
    for row in cursor:
        if "Tribal" in (row[0] or ""):
            row[2] = row[1]
        else:
            row[2] = 0
        cursor.updateRow(row)

print("Tribal funding field added")

Tribal funding field added


#### Check totals

In [166]:
# Newly calculated field totals 
# Fields to sum
fields_to_sum = [
    "LIC_Funding",
    "DAC_Funding",
    "DACN_LIC_Funding",
    "DACN_LIH_Funding",
    "NOPP_Funding",
    "Tribal_Funding"
]

# Dictionary to store the totals
totals = {field: 0 for field in fields_to_sum}

# Iterate over rows and sum values
with arcpy.da.SearchCursor(Funding_Dist_long, fields_to_sum) as cursor:
    for row in cursor:
        for i, field in enumerate(fields_to_sum):
            if row[i] is not None:
                totals[field] += row[i]

# Print results
for field, total in totals.items():
    print(f"{field}: {total/2:,.2f}")

LIC_Funding: 8,575,299,875.36
DAC_Funding: 6,404,868,451.94
DACN_LIC_Funding: 656,089,011.60
DACN_LIH_Funding: 608,268,145.37
NOPP_Funding: 2,745,787,049.97
Tribal_Funding: 35,379,651.77


In [167]:
LIC_designations = {
    "DAC 1/2 mile neighbor: low-income community",
    "Low-income community",
    "Disadvantaged community: CES, Disadvantaged community: Tribal land, Low-income community",
    "Disadvantaged community: CES, Low-income community",
    "Disadvantaged community: Tribal land, Low-income community"
}

DAC_designations = {
    "Disadvantaged community: CES",
    "Disadvantaged community: CES, Disadvantaged community: Tribal land, Low-income community",
    "Disadvantaged community: CES, Low-income community",
    "Disadvantaged community: Tribal land",
    "Disadvantaged community: Tribal land, Low-income community"
}

DACN_LIC_designations = {
    "DAC 1/2 mile neighbor: low-income community"
}

DACN_LIH_designations = {
    "DAC 1/2 mile neighbor: low-income household eligible"
}

NOPP_designations = {
    "Low-income household eligible"
}

tribal_designations = {
    "Disadvantaged community: CES, Disadvantaged community: Tribal land, Low-income community",
    "Disadvantaged community: Tribal land",
    "Disadvantaged community: Tribal land, Low-income community"
}


In [170]:
# Dictionary of designation groups
designation_groups = {
    "LIC": LIC_designations,
    "DAC": DAC_designations,
    "DACN_LIC": DACN_LIC_designations,
    "DACN_LIH": DACN_LIH_designations,
    "NOPP": NOPP_designations,
    "Tribal": tribal_designations
}

# Initialize totals
totals = {key: 0 for key in designation_groups}

# Cursor to iterate through records
with arcpy.da.SearchCursor(Funding_Dist_long, ["designation", "Funding_Amount"]) as cursor:
    for row in cursor:
        designation = row[0]
        amount = row[1] or 0
        for group_name, designation_set in designation_groups.items():
            if designation in designation_set:
                totals[group_name] += amount

# Print the results
for group_name, total in totals.items():
    print(f"{group_name} Total Funding: {total/2:,.2f}")


LIC Total Funding: 8,575,299,875.36
DAC Total Funding: 6,404,868,451.94
DACN_LIC Total Funding: 656,089,011.60
DACN_LIH Total Funding: 608,268,145.37
NOPP Total Funding: 2,745,787,049.97
Tribal Total Funding: 35,379,651.77


### Aggregate by CT in a dataframe

In [ ]:
# Input feature class
fc = Funding_Dist_long

# Convert to pandas DataFrame
fields = [f.name for f in arcpy.ListFields(fc) if f.type not in ("Geometry", "OID")]
data = [row for row in arcpy.da.SearchCursor(fc, fields)]
df = pd.DataFrame(data, columns=fields)

# Grouping and aggregation setup
group_field = ["Census_Tract", "Program_Name"]

fields_to_sum = ["Funding_Amount", "LIC_Funding", "DAC_Funding", "DACN_LIC_Funding", "DACN_LIH_Funding",
    "NOPP_Funding", "Tribal_Funding", "Population"]

fields_to_copy = [
    "PctBIPOC", "BIPOCMaj", "DemogMaj", "CES4range", "CES4decile", 'CCIquartile',
    "County", "ApproxLoc", "Child_10", "Pop_10_64", "Elderly_65", "Hispanic", "White", "African_American",
    "Native_American", "Other_Multiple", "Pacific_Islander", "Asian_American", "CIscoreP", "Ozone_Pctl",
    "PM2_5_Pctl", "Diesel_PM_Pctl", "Pesticides_Pctl", "Tox_Releases_Pctl", "Traffic_Pctl", "DrinkingWaterPctl",
    "Lead_Pctl", "Cleanups_Pctl", "GW_Threats_Pctl", "Haz_Waste_Pctl", "ImpWaterBodPctl", "Solid_Waste_Pctl",
    "Pollution_Pctl", "Asthma_Pctl", "LowBirthW_Pctl", "Cardiovasc_Pctl", "Education_Pctl", "Ling_Isol_Pctl",
    "Poverty_Pctl", "Unemploy_Pctl", "HousBurd_Pctl", "PopCharP", "Tribal_Name", "African_American_Range",
    "Native_American_Range", "Other_Multiple_Range", "Pacific_Islander_Range", "Asian_American_Range", "Hispanic_Range", "White_Range"
]

fields_to_delete = ["CCIpc_prgm", "Geography", "Designatio"]

# Drop fields you don’t want
df_cleaned = df.drop(columns=fields_to_delete, errors='ignore')

# Define aggregation rules
agg_dict = {field: 'sum' for field in fields_to_sum}
for field in fields_to_copy:
    if field == "Name":
        agg_dict[field] = lambda x: next((val for val in x if pd.notnull(val) and str(val).strip() != ""), None)
    else:
        agg_dict[field] = 'first'

# Group and aggregate
funding_CT = df_cleaned.groupby(group_field, as_index=False).agg(agg_dict)

# zero-pad Tract to 11 digits and create GEOID
funding_CT["GEOID"] = funding_CT["Census_Tract"].apply(lambda x: str(int(x)).zfill(11))

funding_CT.sample(10)

,Census_Tract,Program_Name,Funding_Amount,LIC_Funding,DAC_Funding,DACN_LIC_Funding,DACN_LIH_Funding,NOPP_Funding,Tribal_Funding,Population,...,Unemploy_Pctl,HousBurd_Pctl,PopCharP,Tribal_Name,African_American_Range,Native_American_Range,Other_Multiple_Range,Pacific_Islander_Range,Asian_American_Range,GEOID
40458,6.075026e+09,Community Air Protection Incentives,105033.000000,105033.000000,0.0,0.000000,0.00000,0.000000,0.0,5255.0,...,30.882353,21.571610,43.166919,,0-10%,0-10%,0-10%,0-10%,50-60%,06075026302
16552,6.037408e+09,<All Programs>,121847.165934,121847.165934,0.0,22403.295259,0.00000,0.000000,0.0,4986.0,...,55.023425,36.653992,33.648512,,10-20%,0-10%,0-10%,0-10%,30-40%,06037408004
37915,6.073008e+09,Food Waste Prevention and Rescue Grants,750000.000000,0.000000,0.0,0.000000,0.00000,750000.000000,0.0,6979.0,...,9.721499,14.689480,20.902673,,0-10%,0-10%,0-10%,0-10%,50-60%,06073008350
10995,6.037189e+09,Low Carbon Transit Operations Program,6457.377140,0.000000,0.0,0.000000,6457.37714,0.000000,0.0,3356.0,...,45.783446,61.939163,48.928391,,0-10%,0-10%,0-10%,0-10%,0-10%,06037189102
15219,6.037302e+09,Single-Family Solar Photovoltaics (PV),26125.000000,26125.000000,26125.0,0.000000,0.00000,0.000000,0.0,5510.0,...,59.383134,95.754119,82.816440,,0-10%,0-10%,0-10%,0-10%,10-20%,06037302202
33860,6.067008e+09,Water-Energy Grant Program,9.000000,0.000000,0.0,0.000000,0.00000,9.000000,0.0,3059.0,...,66.605934,18.466413,44.654564,,0-10%,0-10%,0-10%,0-10%,0-10%,06067008140
2014,6.001451e+09,Clean Vehicle Rebate Project,527585.000000,0.000000,0.0,0.000000,0.00000,527585.000000,0.0,5630.0,...,1.900052,25.323194,19.578921,,0-10%,0-10%,0-10%,0-10%,50-60%,06001450750
9219,6.037113e+09,<All Programs>,87733.442845,0.000000,0.0,0.000000,0.00000,87733.442845,0.0,2303.0,...,65.629880,75.982256,45.196672,,0-10%,0-10%,10-20%,0-10%,10-20%,06037113231
18492,6.037504e+09,Low Carbon Transit Operations Program,17.862263,17.862263,0.0,0.000000,0.00000,0.000000,0.0,4655.0,...,30.882353,40.899873,45.524458,,0-10%,0-10%,0-10%,0-10%,0-10%,06037503502
19322,6.037535e+09,Clean Cars 4 All,10000.000000,10000.000000,0.0,10000.000000,0.00000,0.000000,0.0,4435.0,...,44.351900,13.561470,69.465456,,0-10%,0-10%,0-10%,0-10%,0-10%,06037534700


In [172]:
#Check the total funding amount to make sure the dataframe has been grouped without error - QC

# Ensure Funding_Amount is numeric
df_cleaned["Funding_Amount"] = pd.to_numeric(df_cleaned["Funding_Amount"], errors='coerce')

# Print number of invalid or missing entries
invalid_count = df_cleaned["Funding_Amount"].isna().sum()
print(f"Invalid or missing Funding_Amount entries: {invalid_count}")

# Fill missing values with 0 (optional, if you want them counted in the sum)
df_cleaned["Funding_Amount"] = df_cleaned["Funding_Amount"].fillna(0)

# Print the total sum
total_funding = df_cleaned["Funding_Amount"].sum()
print(f"Total Funding Amount: {total_funding/2:,.2f}")

Invalid or missing Funding_Amount entries: 0
Total Funding Amount: 12,775,208,202.75


In [173]:
# Export to a CSV (interm out put for census tract code below)

# Define the output CSV file path
output_csv = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Interm Code Outputs\Funding_CT_Long.csv"

# Save the DataFrame to CSV
funding_CT.to_csv(output_csv, index=False)

print(f"Data saved to: {output_csv}")

Data saved to: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Interm Code Outputs\Funding_CT_Long.csv


In [176]:
# Export to an Excel file (final tabular format)

# Define the output Excel file path
output_excel = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Interm Code Outputs\Funding_CT_Long.xlsx"

# Save the DataFrame to Excel
funding_CT.to_excel(output_excel, index=False)

print(f"Data saved to: {output_excel}")

Data saved to: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Interm Code Outputs\Funding_CT_Long.xlsx


### Rejoin to census tract geography

NOTE: due to issues with using Geopandas within the Arc Python enviornment joining the dataset aggregated by census tract is performed in an external enviornment with this notebook: [LINK to GitHub for code]

In [177]:
# Path to your GeoPackage and layer
gpkg_path = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Interm Code Outputs\joined_CT_cci2.gpkg"
layer_name = "main.joined_data"  # Layer inside the GPKG

# Path to your File Geodatabase (use your existing project_gdb variable)
fgdb_path = project_gdb  
# Output feature class name inside the FGDB
out_fc_name = f"Funding_Dist_{suffix}_CT"

# Full path to layer in GPKG
gpkg_layer = f"{gpkg_path}\\{layer_name}"

# Convert feature class from GeoPackage to File Geodatabase
arcpy.FeatureClassToFeatureClass_conversion(gpkg_layer, fgdb_path, out_fc_name)


<Result 'C:\\Users\\gcowan\\Documents\\ArcGIS\\Projects\\Distribution Analysis\\Data Restructure\\Funding_Distribution_AR25\\Funding_Distribution_AR25.gdb\\Funding_Dist_AR25_CT'>

In [178]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.activeMap

# Get the layer by name
layer_name = "Funding_Dist_AR25_CT"
layers = map_obj.listLayers(layer_name)

if not layers:
    print(f"Layer '{layer_name}' not found in the current map.")
else:
    layer = layers[0]  # Safe now
    fc_path = layer.dataSource

    # Fields to delete
    fields_to_delete_CT = [
        'STATEFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD',
        'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
        'GeoID_Fmt', 'GeoID_Fm', 'Tract_x'
    ]

    # Delete fields
    try:
        arcpy.DeleteField_management(fc_path, fields_to_delete_CT)
        print("Fields deleted successfully.")
    except Exception as e:
        print(f"An error occurred during field deletion: {e}")


Fields deleted successfully.


In [179]:
# Define paths
Funding_Dist_CT = f"{project_gdb}\\Funding_Dist_{suffix}_CT"
layer_CT_name = "FeatureLayer_CT"
FundingDist_lpkx_CT = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_CT.lpkx"

try:
    # Step 1: Create a feature layer
    arcpy.management.MakeFeatureLayer(Funding_Dist_CT, layer_CT_name)
    print(f"Feature layer '{layer_CT_name}' created successfully.")

    # Step 1.5: Add to current map
    map_obj = arcpy.mp.ArcGISProject("CURRENT").activeMap
    map_obj.addDataFromPath(Funding_Dist_CT)
    print(f"'{Funding_Dist_CT}' added to map.")

    # Step 2: Create a layer package
    arcpy.management.PackageLayer(layer_CT_name, FundingDist_lpkx_CT, "PRESERVE", "CONVERT_ARCSDE", "DEFAULT", "ALL")
    print(f"Layer package saved successfully at: {FundingDist_lpkx_CT}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Cleanup: Delete the temporary layer
    if arcpy.Exists(layer_CT_name):
        arcpy.management.Delete(layer_CT_name)
        print(f"Temporary layer '{layer_CT_name}' deleted.")


Feature layer 'FeatureLayer_CT' created successfully.
'C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Funding_Distribution_AR25.gdb\Funding_Dist_AR25_CT' added to map.
Layer package saved successfully at: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_CT.lpkx
Temporary layer 'FeatureLayer_CT' deleted.


In [180]:
# Define paths
Funding_Dist_CT = f"{project_gdb}\\Funding_Dist_{suffix}_CT"
layer_CT_name = "FeatureLayer_CT"
FundingDist_lpkx_CT = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_CT.lpkx"

try:
    # Step 1: Create a feature layer
    arcpy.management.MakeFeatureLayer(Funding_Dist_CT, layer_CT_name)
    print(f"Feature layer '{layer_CT_name}' created successfully.")

    # Step 1.5: Add to current map
    map_obj = arcpy.mp.ArcGISProject("CURRENT").activeMap
    map_obj.addDataFromPath(Funding_Dist_CT)
    print(f"'{Funding_Dist_CT}' added to map.")

    # Step 2: Create a layer package
    arcpy.management.PackageLayer(layer_CT_name, FundingDist_lpkx_CT, "PRESERVE", "CONVERT_ARCSDE", "DEFAULT", "ALL")
    print(f"Layer package saved successfully at: {FundingDist_lpkx_CT}")

    # Step 3: Sum the Funding_Amount field
    total = 0
    with arcpy.da.SearchCursor(Funding_Dist_CT, ["Funding_Amount"]) as cursor:
        for row in cursor:
            if row[0] is not None:
                total += row[0]
    print(f"Total Funding_Amount: ${total/2:,.2f}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Cleanup: Delete the temporary layer
    if arcpy.Exists(layer_CT_name):
        arcpy.management.Delete(layer_CT_name)
        print(f"Temporary layer '{layer_CT_name}' deleted.")


Feature layer 'FeatureLayer_CT' created successfully.
'C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Funding_Distribution_AR25.gdb\Funding_Dist_AR25_CT' added to map.
Layer package saved successfully at: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Funding_Distribution_AR25\Final Code Outputs\Funding_Dist_CT.lpkx
Total Funding_Amount: $12,775,208,202.75
Temporary layer 'FeatureLayer_CT' deleted.
